In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW06OT-MakeConcordance-Joshua.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""


import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

Joshua001 = [
    ['Joshua','Jsh','01','001','1',"Now after the death of Moses the servant of the LORD it came to pass, that the LORD spake unto Joshua the son of Nun, Moses' minister, saying,"],
['Joshua','Jsh','01','002','2',"Moses my servant is dead; now therefore arise, go over this Jordan, thou, and all this people, unto the land which I do give to them, \emph{even} to the children of Israel."],
['Joshua','Jsh','01','003','3',"Every place that the sole of your foot shall tread upon, that have I given unto you, as I said unto Moses."],
['Joshua','Jsh','01','004','4',"From the wilderness and this Lebanon even unto the great river, the river Euphrates, all the land of the Hittites, and unto the great sea toward the going down of the sun, shall be your coast."],
['Joshua','Jsh','01','005','5',"There shall not any man be able to stand before thee all the days of thy life: as I was with Moses, \emph{so} I will be with thee: I will not fail thee, nor forsake thee."],
['Joshua','Jsh','01','006','6',"Be strong and of a good courage: for unto this people shalt thou divide for an inheritance the land, which I sware unto their fathers to give them."],
['Joshua','Jsh','01','007','7',"Only be thou strong and very courageous, that thou mayest observe to do according to all the law, which Moses my servant commanded thee: turn not from it \emph{to} the right hand or \emph{to} the left, that thou mayest prosper whithersoever thou goest."],
['Joshua','Jsh','01','008','8',"This book of the law shall not depart out of thy mouth; but thou shalt meditate therein day and night, that thou mayest observe to do according to all that is written therein: for then thou shalt make thy way prosperous, and then thou shalt have good success."],
['Joshua','Jsh','01','009','9',"Have not I commanded thee? Be strong and of a good courage; be not afraid, neither be thou dismayed: for the LORD thy God \emph{is} with thee whithersoever thou goest."],
['Joshua','Jsh','01','010','10',"Then Joshua commanded the officers of the people, saying,"],
['Joshua','Jsh','01','011','11',"Pass through the host, and command the people, saying, Prepare you victuals; for within three days ye shall pass over this Jordan, to go in to possess the land, which the LORD your God giveth you to possess it."],
['Joshua','Jsh','01','012','12',"And to the Reubenites, and to the Gadites, and to half the tribe of Manasseh, spake Joshua, saying,"],
['Joshua','Jsh','01','013','13',"Remember the word which Moses the servant of the LORD commanded you, saying, The LORD your God hath given you rest, and hath given you this land."],
['Joshua','Jsh','01','014','14',"Your wives, your little ones, and your cattle, shall remain in the land which Moses gave you on this side Jordan; but ye shall pass before your brethren armed, all the mighty men of valour, and help them;"],
['Joshua','Jsh','01','015','15',"Until the LORD have given your brethren rest, as \emph{he} \emph{hath} \emph{given} you, and they also have possessed the land which the LORD your God giveth them: then ye shall return unto the land of your possession, and enjoy it, which Moses the LORD'S servant gave you on this side Jordan toward the sunrising."],
['Joshua','Jsh','01','016','16',"And they answered Joshua, saying, All that thou commandest us we will do, and whithersoever thou sendest us, we will go."],
['Joshua','Jsh','01','017','17',"According as we hearkened unto Moses in all things, so will we hearken unto thee: only the LORD thy God be with thee, as he was with Moses."],
['Joshua','Jsh','01','018','18',"Whosoever \emph{he} \emph{be} that doth rebel against thy commandment, and will not hearken unto thy words in all that thou commandest him, he shall be put to death: only be strong and of a good courage."]
]

Joshua002 = [
        ['Joshua','Jsh','02','001','1',"And Joshua the son of Nun sent out of Shittim two men to spy secretly, saying, Go view the land, even Jericho. And they went, and came into an harlot's house, named Rahab, and lodged there."],
['Joshua','Jsh','02','002','2',"And it was told the king of Jericho, saying, Behold, there came men in hither to night of the children of Israel to search out the country."],
['Joshua','Jsh','02','003','3',"And the king of Jericho sent unto Rahab, saying, Bring forth the men that are come to thee, which are entered into thine house: for they be come to search out all the country."],
['Joshua','Jsh','02','004','4',"And the woman took the two men, and hid them, and said thus, There came men unto me, but I wist not whence they \emph{were}:"],
['Joshua','Jsh','02','005','5',"And it came to pass \emph{about} \emph{the} \emph{time} of shutting of the gate, when it was dark, that the men went out: whither the men went I wot not: pursue after them quickly; for ye shall overtake them."],
['Joshua','Jsh','02','006','6',"But she had brought them up to the roof of the house, and hid them with the stalks of flax, which she had laid in order upon the roof."],
['Joshua','Jsh','02','007','7',"And the men pursued after them the way to Jordan unto the fords: and as soon as they which pursued after them were gone out, they shut the gate."],
['Joshua','Jsh','02','008','8',"And before they were laid down, she came up unto them upon the roof;"],
['Joshua','Jsh','02','009','9',"And she said unto the men, I know that the LORD hath given you the land, and that your terror is fallen upon us, and that all the inhabitants of the land faint because of you."],
['Joshua','Jsh','02','010','10',"For we have heard how the LORD dried up the water of the Red sea for you, when ye came out of Egypt; and what ye did unto the two kings of the Amorites, that \emph{were} on the other side Jordan, Sihon and Og, whom ye utterly destroyed."],
['Joshua','Jsh','02','011','11',"And as soon as we had heard \emph{these} \emph{things}, our hearts did melt, neither did there remain any more courage in any man, because of you: for the LORD your God, he \emph{is} God in heaven above, and in earth beneath."],
['Joshua','Jsh','02','012','12',"Now therefore, I pray you, swear unto me by the LORD, since I have shewed you kindness, that ye will also shew kindness unto my father's house, and give me a true token:"],
['Joshua','Jsh','02','013','13',"And \emph{that} ye will save alive my father, and my mother, and my brethren, and my sisters, and all that they have, and deliver our lives from death."],
['Joshua','Jsh','02','014','14',"And the men answered her, Our life for your's, if ye utter not this our business. And it shall be, when the LORD hath given us the land, that we will deal kindly and truly with thee."],
['Joshua','Jsh','02','015','15',"Then she let them down by a cord through the window: for her house \emph{was} upon the town wall, and she dwelt upon the wall."],
['Joshua','Jsh','02','016','16',"And she said unto them, Get you to the mountain, lest the pursuers meet you; and hide yourselves there three days, until the pursuers be returned: and afterward may ye go your way."],
['Joshua','Jsh','02','017','17',"And the men said unto her, We \emph{will} \emph{be} blameless of this thine oath which thou hast made us swear."],
['Joshua','Jsh','02','018','18',"Behold, \emph{when} we come into the land, thou shalt bind this line of scarlet thread in the window which thou didst let us down by: and thou shalt bring thy father, and thy mother, and thy brethren, and all thy father's household, home unto thee."],
['Joshua','Jsh','02','019','19',"And it shall be, \emph{that} whosoever shall go out of the doors of thy house into the street, his blood \emph{shall} \emph{be} upon his head, and we \emph{will} \emph{be} guiltless: and whosoever shall be with thee in the house, his blood \emph{shall} \emph{be} on our head, if \emph{any} hand be upon him."],
['Joshua','Jsh','02','020','20',"And if thou utter this our business, then we will be quit of thine oath which thou hast made us to swear."],
['Joshua','Jsh','02','021','21',"And she said, According unto your words, so \emph{be} it. And she sent them away, and they departed: and she bound the scarlet line in the window."],
['Joshua','Jsh','02','022','22',"And they went, and came unto the mountain, and abode there three days, until the pursuers were returned: and the pursuers sought \emph{them} throughout all the way, but found \emph{them} not."],
['Joshua','Jsh','02','023','23',"So the two men returned, and descended from the mountain, and passed over, and came to Joshua the son of Nun, and told him all \emph{things} that befell them:"],
['Joshua','Jsh','02','024','24',"And they said unto Joshua, Truly the LORD hath delivered into our hands all the land; for even all the inhabitants of the country do faint because of us."]]

Joshua003 = [
    ['Joshua','Jsh','03','001','1',"And Joshua rose early in the morning; and they removed from Shittim, and came to Jordan, he and all the children of Israel, and lodged there before they passed over."],
['Joshua','Jsh','03','002','2',"And it came to pass after three days, that the officers went through the host;"],
['Joshua','Jsh','03','003','3',"And they commanded the people, saying, When ye see the ark of the covenant of the LORD your God, and the priests the Levites bearing it, then ye shall remove from your place, and go after it."],
['Joshua','Jsh','03','004','4',"Yet there shall be a space between you and it, about two thousand cubits by measure: come not near unto it, that ye may know the way by which ye must go: for ye have not passed \emph{this} way heretofore."],
['Joshua','Jsh','03','005','5',"And Joshua said unto the people, Sanctify yourselves: for to morrow the LORD will do wonders among you."],
['Joshua','Jsh','03','006','6',"And Joshua spake unto the priests, saying, Take up the ark of the covenant, and pass over before the people. And they took up the ark of the covenant, and went before the people."],
['Joshua','Jsh','03','007','7',"And the LORD said unto Joshua, This day will I begin to magnify thee in the sight of all Israel, that they may know that, as I was with Moses, \emph{so} I will be with thee."],
['Joshua','Jsh','03','008','8',"And thou shalt command the priests that bear the ark of the covenant, saying, When ye are come to the brink of the water of Jordan, ye shall stand still in Jordan."],
['Joshua','Jsh','03','009','9',"And Joshua said unto the children of Israel, Come hither, and hear the words of the LORD your God."],
['Joshua','Jsh','03','010','10',"And Joshua said, Hereby ye shall know that the living God \emph{is} among you, and \emph{that} he will without fail drive out from before you the Canaanites, and the Hittites, and the Hivites, and the Perizzites, and the Girgashites, and the Amorites, and the Jebusites."],
['Joshua','Jsh','03','011','11',"Behold, the ark of the covenant of the Lord of all the earth passeth over before you into Jordan."],
['Joshua','Jsh','03','012','12',"Now therefore take you twelve men out of the tribes of Israel, out of every tribe a man."],
['Joshua','Jsh','03','013','13',"And it shall come to pass, as soon as the soles of the feet of the priests that bear the ark of the LORD, the Lord of all the earth, shall rest in the waters of Jordan, \emph{that} the waters of Jordan shall be cut off \emph{from} the waters that come down from above; and they shall stand upon an heap."],
['Joshua','Jsh','03','014','14',"And it came to pass, when the people removed from their tents, to pass over Jordan, and the priests bearing the ark of the covenant before the people;"],
['Joshua','Jsh','03','015','15',"And as they that bare the ark were come unto Jordan, and the feet of the priests that bare the ark were dipped in the brim of the water, (for Jordan overfloweth all his banks all the time of harvest,)"],
['Joshua','Jsh','03','016','16',"That the waters which came down from above stood \emph{and} rose up upon an heap very far from the city Adam, that \emph{is} beside Zaretan: and those that came down toward the sea of the plain, \emph{even} the salt sea, failed, \emph{and} were cut off: and the people passed over right against Jericho."],
['Joshua','Jsh','03','017','17',"And the priests that bare the ark of the covenant of the LORD stood firm on dry ground in the midst of Jordan, and all the Israelites passed over on dry ground, until all the people were passed clean over Jordan."]]

Joshua004 = [
    ['Joshua','Jsh','04','001','1',"And it came to pass, when all the people were clean passed over Jordan, that the LORD spake unto Joshua, saying,"],
['Joshua','Jsh','04','002','2',"Take you twelve men out of the people, out of every tribe a man,"],
['Joshua','Jsh','04','003','3',"And command ye them, saying, Take you hence out of the midst of Jordan, out of the place where the priests' feet stood firm, twelve stones, and ye shall carry them over with you, and leave them in the lodging place, where ye shall lodge this night."],
['Joshua','Jsh','04','004','4',"Then Joshua called the twelve men, whom he had prepared of the children of Israel, out of every tribe a man:"],
['Joshua','Jsh','04','005','5',"And Joshua said unto them, Pass over before the ark of the LORD your God into the midst of Jordan, and take ye up every man of you a stone upon his shoulder, according unto the number of the tribes of the children of Israel:"],
['Joshua','Jsh','04','006','6',"That this may be a sign among you, \emph{that} when your children ask \emph{their} \emph{fathers} in time to come, saying, What \emph{mean} ye by these stones?"],
['Joshua','Jsh','04','007','7',"Then ye shall answer them, That the waters of Jordan were cut off before the ark of the covenant of the LORD; when it passed over Jordan, the waters of Jordan were cut off: and these stones shall be for a memorial unto the children of Israel for ever."],
['Joshua','Jsh','04','008','8',"And the children of Israel did so as Joshua commanded, and took up twelve stones out of the midst of Jordan, as the LORD spake unto Joshua, according to the number of the tribes of the children of Israel, and carried them over with them unto the place where they lodged, and laid them down there."],
['Joshua','Jsh','04','009','9',"And Joshua set up twelve stones in the midst of Jordan, in the place where the feet of the priests which bare the ark of the covenant stood: and they are there unto this day."],
['Joshua','Jsh','04','010','10',"For the priests which bare the ark stood in the midst of Jordan, until every thing was finished that the LORD commanded Joshua to speak unto the people, according to all that Moses commanded Joshua: and the people hasted and passed over."],
['Joshua','Jsh','04','011','11',"And it came to pass, when all the people were clean passed over, that the ark of the LORD passed over, and the priests, in the presence of the people."],
['Joshua','Jsh','04','012','12',"And the children of Reuben, and the children of Gad, and half the tribe of Manasseh, passed over armed before the children of Israel, as Moses spake unto them:"],
['Joshua','Jsh','04','013','13',"About forty thousand prepared for war passed over before the LORD unto battle, to the plains of Jericho."],
['Joshua','Jsh','04','014','14',"On that day the LORD magnified Joshua in the sight of all Israel; and they feared him, as they feared Moses, all the days of his life."],
['Joshua','Jsh','04','015','15',"And the LORD spake unto Joshua, saying,"],
['Joshua','Jsh','04','016','16',"Command the priests that bear the ark of the testimony, that they come up out of Jordan."],
['Joshua','Jsh','04','017','17',"Joshua therefore commanded the priests, saying, Come ye up out of Jordan."],
['Joshua','Jsh','04','018','18',"And it came to pass, when the priests that bare the ark of the covenant of the LORD were come up out of the midst of Jordan, \emph{and} the soles of the priests' feet were lifted up unto the dry land, that the waters of Jordan returned unto their place, and flowed over all his banks, as \emph{they} \emph{did} before."],
['Joshua','Jsh','04','019','19',"And the people came up out of Jordan on the tenth \emph{day} of the first month, and encamped in Gilgal, in the east border of Jericho."],
['Joshua','Jsh','04','020','20',"And those twelve stones, which they took out of Jordan, did Joshua pitch in Gilgal."],
['Joshua','Jsh','04','021','21',"And he spake unto the children of Israel, saying, When your children shall ask their fathers in time to come, saying, What \emph{mean} these stones?"],
['Joshua','Jsh','04','022','22',"Then ye shall let your children know, saying, Israel came over this Jordan on dry land."],
['Joshua','Jsh','04','023','23',"For the LORD your God dried up the waters of Jordan from before you, until ye were passed over, as the LORD your God did to the Red sea, which he dried up from before us, until we were gone over:"],
['Joshua','Jsh','04','024','24',"That all the people of the earth might know the hand of the LORD, that it \emph{is} mighty: that ye might fear the LORD your God for ever."]
]

Joshua005 = [
    ['Joshua','Jsh','05','001','1',"And it came to pass, when all the kings of the Amorites, which \emph{were} on the side of Jordan westward, and all the kings of the Canaanites, which \emph{were} by the sea, heard that the LORD had dried up the waters of Jordan from before the children of Israel, until we were passed over, that their heart melted, neither was there spirit in them any more, because of the children of Israel."],
['Joshua','Jsh','05','002','2',"At that time the LORD said unto Joshua, Make thee sharp knives, and circumcise again the children of Israel the second time."],
['Joshua','Jsh','05','003','3',"And Joshua made him sharp knives, and circumcised the children of Israel at the hill of the foreskins."],
['Joshua','Jsh','05','004','4',"And this \emph{is} the cause why Joshua did circumcise: All the people that came out of Egypt, \emph{that} \emph{were} males, \emph{even} all the men of war, died in the wilderness by the way, after they came out of Egypt."],
['Joshua','Jsh','05','005','5',"Now all the people that came out were circumcised: but all the people \emph{that} \emph{were} born in the wilderness by the way as they came forth out of Egypt, \emph{them} they had not circumcised."],
['Joshua','Jsh','05','006','6',"For the children of Israel walked forty years in the wilderness, till all the people \emph{that} \emph{were} men of war, which came out of Egypt, were consumed, because they obeyed not the voice of the LORD: unto whom the LORD sware that he would not shew them the land, which the LORD sware unto their fathers that he would give us, a land that floweth with milk and honey."],
['Joshua','Jsh','05','007','7',"And their children, \emph{whom} he raised up in their stead, them Joshua circumcised: for they were uncircumcised, because they had not circumcised them by the way."],
['Joshua','Jsh','05','008','8',"And it came to pass, when they had done circumcising all the people, that they abode in their places in the camp, till they were whole."],
['Joshua','Jsh','05','009','9',"And the LORD said unto Joshua, This day have I rolled away the reproach of Egypt from off you. Wherefore the name of the place is called Gilgal unto this day."],
['Joshua','Jsh','05','010','10',"And the children of Israel encamped in Gilgal, and kept the passover on the fourteenth day of the month at even in the plains of Jericho."],
['Joshua','Jsh','05','011','11',"And they did eat of the old corn of the land on the morrow after the passover, unleavened cakes, and parched \emph{corn} in the selfsame day."],
['Joshua','Jsh','05','012','12',"And the manna ceased on the morrow after they had eaten of the old corn of the land; neither had the children of Israel manna any more; but they did eat of the fruit of the land of Canaan that year."],
['Joshua','Jsh','05','013','13',"And it came to pass, when Joshua was by Jericho, that he lifted up his eyes and looked, and, behold, there stood a man over against him with his sword drawn in his hand: and Joshua went unto him, and said unto him, \emph{Art} thou for us, or for our adversaries?"],
['Joshua','Jsh','05','014','14',"And he said, Nay; but \emph{as} captain of the host of the LORD am I now come. And Joshua fell on his face to the earth, and did worship, and said unto him, What saith my lord unto his servant?"],
['Joshua','Jsh','05','015','15',"And the captain of the LORD'S host said unto Joshua, Loose thy shoe from off thy foot; for the place whereon thou standest \emph{is} holy. And Joshua did so."]
]

Joshua006 = [
    ['Joshua','Jsh','06','001','1',"Now Jericho was straitly shut up because of the children of Israel: none went out, and none came in."],
['Joshua','Jsh','06','002','2',"And the LORD said unto Joshua, See, I have given into thine hand Jericho, and the king thereof, \emph{and} the mighty men of valour."],
['Joshua','Jsh','06','003','3',"And ye shall compass the city, all \emph{ye} men of war, \emph{and} go round about the city once. Thus shalt thou do six days."],
['Joshua','Jsh','06','004','4',"And seven priests shall bear before the ark seven trumpets of rams' horns: and the seventh day ye shall compass the city seven times, and the priests shall blow with the trumpets."],
['Joshua','Jsh','06','005','5',"And it shall come to pass, that when they make a long \emph{blast} with the ram's horn, \emph{and} when ye hear the sound of the trumpet, all the people shall shout with a great shout; and the wall of the city shall fall down flat, and the people shall ascend up every man straight before him."],
['Joshua','Jsh','06','006','6',"And Joshua the son of Nun called the priests, and said unto them, Take up the ark of the covenant, and let seven priests bear seven trumpets of rams' horns before the ark of the LORD."],
['Joshua','Jsh','06','007','7',"And he said unto the people, Pass on, and compass the city, and let him that is armed pass on before the ark of the LORD."],
['Joshua','Jsh','06','008','8',"And it came to pass, when Joshua had spoken unto the people, that the seven priests bearing the seven trumpets of rams' horns passed on before the LORD, and blew with the trumpets: and the ark of the covenant of the LORD followed them."],
['Joshua','Jsh','06','009','9',"And the armed men went before the priests that blew with the trumpets, and the rereward came after the ark, \emph{the} \emph{priests} going on, and blowing with the trumpets."],
['Joshua','Jsh','06','010','10',"And Joshua had commanded the people, saying, Ye shall not shout, nor make any noise with your voice, neither shall \emph{any} word proceed out of your mouth, until the day I bid you shout; then shall ye shout."],
['Joshua','Jsh','06','011','11',"So the ark of the LORD compassed the city, going about \emph{it} once: and they came into the camp, and lodged in the camp."],
['Joshua','Jsh','06','012','12',"And Joshua rose early in the morning, and the priests took up the ark of the LORD."],
['Joshua','Jsh','06','013','13',"And seven priests bearing seven trumpets of rams' horns before the ark of the LORD went on continually, and blew with the trumpets: and the armed men went before them; but the rereward came after the ark of the LORD, \emph{the} \emph{priests} going on, and blowing with the trumpets."],
['Joshua','Jsh','06','014','14',"And the second day they compassed the city once, and returned into the camp: so they did six days."],
['Joshua','Jsh','06','015','15',"And it came to pass on the seventh day, that they rose early about the dawning of the day, and compassed the city after the same manner seven times: only on that day they compassed the city seven times."],
['Joshua','Jsh','06','016','16',"And it came to pass at the seventh time, when the priests blew with the trumpets, Joshua said unto the people, Shout; for the LORD hath given you the city."],
['Joshua','Jsh','06','017','17',"And the city shall be accursed, \emph{even} it, and all that \emph{are} therein, to the LORD: only Rahab the harlot shall live, she and all that \emph{are} with her in the house, because she hid the messengers that we sent."],
['Joshua','Jsh','06','018','18',"And ye, in any wise keep \emph{yourselves} from the accursed thing, lest ye make \emph{yourselves} accursed, when ye take of the accursed thing, and make the camp of Israel a curse, and trouble it."],
['Joshua','Jsh','06','019','19',"But all the silver, and gold, and vessels of brass and iron, \emph{are} consecrated unto the LORD: they shall come into the treasury of the LORD."],
['Joshua','Jsh','06','020','20',"So the people shouted when \emph{the} \emph{priests} blew with the trumpets: and it came to pass, when the people heard the sound of the trumpet, and the people shouted with a great shout, that the wall fell down flat, so that the people went up into the city, every man straight before him, and they took the city."],
['Joshua','Jsh','06','021','21',"And they utterly destroyed all that \emph{was} in the city, both man and woman, young and old, and ox, and sheep, and ass, with the edge of the sword."],
['Joshua','Jsh','06','022','22',"But Joshua had said unto the two men that had spied out the country, Go into the harlot's house, and bring out thence the woman, and all that she hath, as ye sware unto her."],
['Joshua','Jsh','06','023','23',"And the young men that were spies went in, and brought out Rahab, and her father, and her mother, and her brethren, and all that she had; and they brought out all her kindred, and left them without the camp of Israel."],
['Joshua','Jsh','06','024','24',"And they burnt the city with fire, and all that \emph{was} therein: only the silver, and the gold, and the vessels of brass and of iron, they put into the treasury of the house of the LORD."],
['Joshua','Jsh','06','025','25',"And Joshua saved Rahab the harlot alive, and her father's household, and all that she had; and she dwelleth in Israel \emph{even} unto this day; because she hid the messengers, which Joshua sent to spy out Jericho."],
['Joshua','Jsh','06','026','26',"And Joshua adjured \emph{them} at that time, saying, Cursed \emph{be} the man before the LORD, that riseth up and buildeth this city Jericho: he shall lay the foundation thereof in his firstborn, and in his youngest \emph{son} shall he set up the gates of it."],
['Joshua','Jsh','06','027','27',"So the LORD was with Joshua; and his fame was \emph{noised} throughout all the country."]
]

Joshua007 = [
    ['Joshua','Jsh','07','001','1',"Then Jerubbaal, who \emph{is} Gideon, and all the people that \emph{were} with him, rose up early, and pitched beside the well of Harod: so that the host of the Midianites were on the north side of them, by the hill of Moreh, in the valley."],
['Joshua','Jsh','07','002','2',"And the LORD said unto Gideon, The people that \emph{are} with thee \emph{are} too many for me to give the Midianites into their hands, lest Israel vaunt themselves against me, saying, Mine own hand hath saved me."],
['Joshua','Jsh','07','003','3',"Now therefore go to, proclaim in the ears of the people, saying, Whosoever \emph{is} fearful and afraid, let him return and depart early from mount Gilead. And there returned of the people twenty and two thousand; and there remained ten thousand."],
['Joshua','Jsh','07','004','4',"And the LORD said unto Gideon, The people \emph{are} yet \emph{too} many; bring them down unto the water, and I will try them for thee there: and it shall be, \emph{that} of whom I say unto thee, This shall go with thee, the same shall go with thee; and of whomsoever I say unto thee, This shall not go with thee, the same shall not go."],
['Joshua','Jsh','07','005','5',"So he brought down the people unto the water: and the LORD said unto Gideon, Every one that lappeth of the water with his tongue, as a dog lappeth, him shalt thou set by himself; likewise every one that boweth down upon his knees to drink."],
['Joshua','Jsh','07','006','6',"And the number of them that lapped, \emph{putting} their hand to their mouth, were three hundred men: but all the rest of the people bowed down upon their knees to drink water."],
['Joshua','Jsh','07','007','7',"And the LORD said unto Gideon, By the three hundred men that lapped will I save you, and deliver the Midianites into thine hand: and let all the \emph{other} people go every man unto his place."],
['Joshua','Jsh','07','008','8',"So the people took victuals in their hand, and their trumpets: and he sent all \emph{the} \emph{rest} \emph{of} Israel every man unto his tent, and retained those three hundred men: and the host of Midian was beneath him in the valley."],
['Joshua','Jsh','07','009','9',"And it came to pass the same night, that the LORD said unto him, Arise, get thee down unto the host; for I have delivered it into thine hand."],
['Joshua','Jsh','07','010','10',"But if thou fear to go down, go thou with Phurah thy servant down to the host:"],
['Joshua','Jsh','07','011','11',"And thou shalt hear what they say; and afterward shall thine hands be strengthened to go down unto the host. Then went he down with Phurah his servant unto the outside of the armed men that \emph{were} in the host."],
['Joshua','Jsh','07','012','12',"And the Midianites and the Amalekites and all the children of the east lay along in the valley like grasshoppers for multitude; and their camels \emph{were} without number, as the sand by the sea side for multitude."],
['Joshua','Jsh','07','013','13',"And when Gideon was come, behold, \emph{there} \emph{was} a man that told a dream unto his fellow, and said, Behold, I dreamed a dream, and, lo, a cake of barley bread tumbled into the host of Midian, and came unto a tent, and smote it that it fell, and overturned it, that the tent lay along."],
['Joshua','Jsh','07','014','14',"And his fellow answered and said, This \emph{is} nothing else save the sword of Gideon the son of Joash, a man of Israel: \emph{for} into his hand hath God delivered Midian, and all the host."],
['Joshua','Jsh','07','015','15',"And it was \emph{so}, when Gideon heard the telling of the dream, and the interpretation thereof, that he worshipped, and returned into the host of Israel, and said, Arise; for the LORD hath delivered into your hand the host of Midian."],
['Joshua','Jsh','07','016','16',"And he divided the three hundred men \emph{into} three companies, and he put a trumpet in every man's hand, with empty pitchers, and lamps within the pitchers."],
['Joshua','Jsh','07','017','17',"And he said unto them, Look on me, and do likewise: and, behold, when I come to the outside of the camp, it shall be \emph{that}, as I do, so shall ye do."],
['Joshua','Jsh','07','018','18',"When I blow with a trumpet, I and all that \emph{are} with me, then blow ye the trumpets also on every side of all the camp, and say, \emph{The} \emph{sword} of the LORD, and of Gideon."],
['Joshua','Jsh','07','019','19',"So Gideon, and the hundred men that \emph{were} with him, came unto the outside of the camp in the beginning of the middle watch; and they had but newly set the watch: and they blew the trumpets, and brake the pitchers that \emph{were} in their hands."],
['Joshua','Jsh','07','020','20',"And the three companies blew the trumpets, and brake the pitchers, and held the lamps in their left hands, and the trumpets in their right hands to blow \emph{withal}: and they cried, The sword of the LORD, and of Gideon."],
['Joshua','Jsh','07','021','21',"And they stood every man in his place round about the camp: and all the host ran, and cried, and fled."],
['Joshua','Jsh','07','022','22',"And the three hundred blew the trumpets, and the LORD set every man's sword against his fellow, even throughout all the host: and the host fled to Beth-shittah in Zererath, \emph{and} to the border of Abel-meholah, unto Tabbath."],
['Joshua','Jsh','07','023','23',"And the men of Israel gathered themselves together out of Naphtali, and out of Asher, and out of all Manasseh, and pursued after the Midianites."],
['Joshua','Jsh','07','024','24',"And Gideon sent messengers throughout all mount Ephraim, saying, Come down against the Midianites, and take before them the waters unto Beth-barah and Jordan. Then all the men of Ephraim gathered themselves together, and took the waters unto Beth-barah and Jordan."],
['Joshua','Jsh','07','025','25',"And they took two princes of the Midianites, Oreb and Zeeb; and they slew Oreb upon the rock Oreb, and Zeeb they slew at the winepress of Zeeb, and pursued Midian, and brought the heads of Oreb and Zeeb to Gideon on the other side Jordan."],
['Joshua','Jsh','07','026','26',"And they raised over him a great heap of stones unto this day. So the LORD turned from the fierceness of his anger. Wherefore the name of that place was called, The valley of Achor, unto this day."]
]

Joshua008 = [
    ['Joshua','Jsh','08','001','1',"And the LORD said unto Joshua, Fear not, neither be thou dismayed: take all the people of war with thee, and arise, go up to Ai: see, I have given into thy hand the king of Ai, and his people, and his city, and his land:"],
['Joshua','Jsh','08','002','2',"And thou shalt do to Ai and her king as thou didst unto Jericho and her king: only the spoil thereof, and the cattle thereof, shall ye take for a prey unto yourselves: lay thee an ambush for the city behind it."],
['Joshua','Jsh','08','003','3',"So Joshua arose, and all the people of war, to go up against Ai: and Joshua chose out thirty thousand mighty men of valour, and sent them away by night."],
['Joshua','Jsh','08','004','4',"And he commanded them, saying, Behold, ye shall lie in wait against the city, \emph{even} behind the city: go not very far from the city, but be ye all ready:"],
['Joshua','Jsh','08','005','5',"And I, and all the people that \emph{are} with me, will approach unto the city: and it shall come to pass, when they come out against us, as at the first, that we will flee before them,"],
['Joshua','Jsh','08','006','6',"(For they will come out after us) till we have drawn them from the city; for they will say, They flee before us, as at the first: therefore we will flee before them."],
['Joshua','Jsh','08','007','7',"Then ye shall rise up from the ambush, and seize upon the city: for the LORD your God will deliver it into your hand."],
['Joshua','Jsh','08','008','8',"And it shall be, when ye have taken the city, \emph{that} ye shall set the city on fire: according to the commandment of the LORD shall ye do. See, I have commanded you."],
['Joshua','Jsh','08','009','9',"Joshua therefore sent them forth: and they went to lie in ambush, and abode between Beth-el and Ai, on the west side of Ai: but Joshua lodged that night among the people."],
['Joshua','Jsh','08','010','10',"And Joshua rose up early in the morning, and numbered the people, and went up, he and the elders of Israel, before the people to Ai."],
['Joshua','Jsh','08','011','11',"And all the people, \emph{even} \emph{the} \emph{people} of war that \emph{were} with him, went up, and drew nigh, and came before the city, and pitched on the north side of Ai: now \emph{there} \emph{was} a valley between them and Ai."],
['Joshua','Jsh','08','012','12',"And he took about five thousand men, and set them to lie in ambush between Beth-el and Ai, on the west side of the city."],
['Joshua','Jsh','08','013','13',"And when they had set the people, \emph{even} all the host that \emph{was} on the north of the city, and their liers in wait on the west of the city, Joshua went that night into the midst of the valley."],
['Joshua','Jsh','08','014','14',"And it came to pass, when the king of Ai saw \emph{it}, that they hasted and rose up early, and the men of the city went out against Israel to battle, he and all his people, at a time appointed, before the plain; but he wist not that \emph{there} \emph{were} liers in ambush against him behind the city."],
['Joshua','Jsh','08','015','15',"And Joshua and all Israel made as if they were beaten before them, and fled by the way of the wilderness."],
['Joshua','Jsh','08','016','16',"And all the people that \emph{were} in Ai were called together to pursue after them: and they pursued after Joshua, and were drawn away from the city."],
['Joshua','Jsh','08','017','17',"And there was not a man left in Ai or Beth-el, that went not out after Israel: and they left the city open, and pursued after Israel."],
['Joshua','Jsh','08','018','18',"And the LORD said unto Joshua, Stretch out the spear that \emph{is} in thy hand toward Ai; for I will give it into thine hand. And Joshua stretched out the spear that \emph{he} \emph{had} in his hand toward the city."],
['Joshua','Jsh','08','019','19',"And the ambush arose quickly out of their place, and they ran as soon as he had stretched out his hand: and they entered into the city, and took it, and hasted and set the city on fire."],
['Joshua','Jsh','08','020','20',"And when the men of Ai looked behind them, they saw, and, behold, the smoke of the city ascended up to heaven, and they had no power to flee this way or that way: and the people that fled to the wilderness turned back upon the pursuers."],
['Joshua','Jsh','08','021','21',"And when Joshua and all Israel saw that the ambush had taken the city, and that the smoke of the city ascended, then they turned again, and slew the men of Ai."],
['Joshua','Jsh','08','022','22',"And the other issued out of the city against them; so they were in the midst of Israel, some on this side, and some on that side: and they smote them, so that they let none of them remain or escape."],
['Joshua','Jsh','08','023','23',"And the king of Ai they took alive, and brought him to Joshua."],
['Joshua','Jsh','08','024','24',"And it came to pass, when Israel had made an end of slaying all the inhabitants of Ai in the field, in the wilderness wherein they chased them, and when they were all fallen on the edge of the sword, until they were consumed, that all the Israelites returned unto Ai, and smote it with the edge of the sword."],
['Joshua','Jsh','08','025','25',"And \emph{so} it was, \emph{that} all that fell that day, both of men and women, \emph{were} twelve thousand, \emph{even} all the men of Ai."],
['Joshua','Jsh','08','026','26',"For Joshua drew not his hand back, wherewith he stretched out the spear, until he had utterly destroyed all the inhabitants of Ai."],
['Joshua','Jsh','08','027','27',"Only the cattle and the spoil of that city Israel took for a prey unto themselves, according unto the word of the LORD which he commanded Joshua."],
['Joshua','Jsh','08','028','28',"And Joshua burnt Ai, and made it an heap for ever, \emph{even} a desolation unto this day."],
['Joshua','Jsh','08','029','29',"And the king of Ai he hanged on a tree until eventide: and as soon as the sun was down, Joshua commanded that they should take his carcase down from the tree, and cast it at the entering of the gate of the city, and raise thereon a great heap of stones, \emph{that} \emph{remaineth} unto this day."],
['Joshua','Jsh','08','030','30',"Then Joshua built an altar unto the LORD God of Israel in mount Ebal,"],
['Joshua','Jsh','08','031','31',"As Moses the servant of the LORD commanded the children of Israel, as it is written in the book of the law of Moses, an altar of whole stones, over which no man hath lift up \emph{any} iron: and they offered thereon burnt offerings unto the LORD, and sacrificed peace offerings."],
['Joshua','Jsh','08','032','32',"And he wrote there upon the stones a copy of the law of Moses, which he wrote in the presence of the children of Israel."],
['Joshua','Jsh','08','033','33',"And all Israel, and their elders, and officers, and their judges, stood on this side the ark and on that side before the priests the Levites, which bare the ark of the covenant of the LORD, as well the stranger, as he that was born among them; half of them over against mount Gerizim, and half of them over against mount Ebal; as Moses the servant of the LORD had commanded before, that they should bless the people of Israel."],
['Joshua','Jsh','08','034','34',"And afterward he read all the words of the law, the blessings and cursings, according to all that is written in the book of the law."],
['Joshua','Jsh','08','035','35',"There was not a word of all that Moses commanded, which Joshua read not before all the congregation of Israel, with the women, and the little ones, and the strangers that were conversant among them."]
]

Joshua009 = [
    ['Joshua','Jsh','09','001','1',"And it came to pass, when all the kings which \emph{were} on this side Jordan, in the hills, and in the valleys, and in all the coasts of the great sea over against Lebanon, the Hittite, and the Amorite, the Canaanite, the Perizzite, the Hivite, and the Jebusite, heard \emph{thereof};"],
['Joshua','Jsh','09','002','2',"That they gathered themselves together, to fight with Joshua and with Israel, with one accord."],
['Joshua','Jsh','09','003','3',"And when the inhabitants of Gibeon heard what Joshua had done unto Jericho and to Ai,"],
['Joshua','Jsh','09','004','4',"They did work wilily, and went and made as if they had been ambassadors, and took old sacks upon their asses, and wine bottles, old, and rent, and bound up;"],
['Joshua','Jsh','09','005','5',"And old shoes and clouted upon their feet, and old garments upon them; and all the bread of their provision was dry \emph{and} mouldy."],
['Joshua','Jsh','09','006','6',"And they went to Joshua unto the camp at Gilgal, and said unto him, and to the men of Israel, We be come from a far country: now therefore make ye a league with us."],
['Joshua','Jsh','09','007','7',"And the men of Israel said unto the Hivites, Peradventure ye dwell among us; and how shall we make a league with you?"],
['Joshua','Jsh','09','008','8',"And they said unto Joshua, We \emph{are} thy servants. And Joshua said unto them, Who \emph{are} ye? and from whence come ye?"],
['Joshua','Jsh','09','009','9',"And they said unto him, From a very far country thy servants are come because of the name of the LORD thy God: for we have heard the fame of him, and all that he did in Egypt,"],
['Joshua','Jsh','09','010','10',"And all that he did to the two kings of the Amorites, that \emph{were} beyond Jordan, to Sihon king of Heshbon, and to Og king of Bashan, which \emph{was} at Ashtaroth."],
['Joshua','Jsh','09','011','11',"Wherefore our elders and all the inhabitants of our country spake to us, saying, Take victuals with you for the journey, and go to meet them, and say unto them, We \emph{are} your servants: therefore now make ye a league with us."],
['Joshua','Jsh','09','012','12',"This our bread we took hot \emph{for} our provision out of our houses on the day we came forth to go unto you; but now, behold, it is dry, and it is mouldy:"],
['Joshua','Jsh','09','013','13',"And these bottles of wine, which we filled, \emph{were} new; and, behold, they be rent: and these our garments and our shoes are become old by reason of the very long journey."],
['Joshua','Jsh','09','014','14',"And the men took of their victuals, and asked not \emph{counsel} at the mouth of the LORD."],
['Joshua','Jsh','09','015','15',"And Joshua made peace with them, and made a league with them, to let them live: and the princes of the congregation sware unto them."],
['Joshua','Jsh','09','016','16',"And it came to pass at the end of three days after they had made a league with them, that they heard that they \emph{were} their neighbours, and \emph{that} they dwelt among them."],
['Joshua','Jsh','09','017','17',"And the children of Israel journeyed, and came unto their cities on the third day. Now their cities \emph{were} Gibeon, and Chephirah, and Beeroth, and Kirjath-jearim."],
['Joshua','Jsh','09','018','18',"And the children of Israel smote them not, because the princes of the congregation had sworn unto them by the LORD God of Israel. And all the congregation murmured against the princes."],
['Joshua','Jsh','09','019','19',"But all the princes said unto all the congregation, We have sworn unto them by the LORD God of Israel: now therefore we may not touch them."],
['Joshua','Jsh','09','020','20',"This we will do to them; we will even let them live, lest wrath be upon us, because of the oath which we sware unto them."],
['Joshua','Jsh','09','021','21',"And the princes said unto them, Let them live; but let them be hewers of wood and drawers of water unto all the congregation; as the princes had promised them."],
['Joshua','Jsh','09','022','22',"And Joshua called for them, and he spake unto them, saying, Wherefore have ye beguiled us, saying, We \emph{are} very far from you; when ye dwell among us?"],
['Joshua','Jsh','09','023','23',"Now therefore ye \emph{are} cursed, and there shall none of you be freed from being bondmen, and hewers of wood and drawers of water for the house of my God."],
['Joshua','Jsh','09','024','24',"And they answered Joshua, and said, Because it was certainly told thy servants, how that the LORD thy God commanded his servant Moses to give you all the land, and to destroy all the inhabitants of the land from before you, therefore we were sore afraid of our lives because of you, and have done this thing."],
['Joshua','Jsh','09','025','25',"And now, behold, we \emph{are} in thine hand: as it seemeth good and right unto thee to do unto us, do."],
['Joshua','Jsh','09','026','26',"And so did he unto them, and delivered them out of the hand of the children of Israel, that they slew them not."],
['Joshua','Jsh','09','027','27',"And Joshua made them that day hewers of wood and drawers of water for the congregation, and for the altar of the LORD, even unto this day, in the place which he should choose."]
]

Joshua010 = [
    ['Joshua','Jsh','10','001','1',"Now it came to pass, when Adoni-zedek king of Jerusalem had heard how Joshua had taken Ai, and had utterly destroyed it; as he had done to Jericho and her king, so he had done to Ai and her king; and how the inhabitants of Gibeon had made peace with Israel, and were among them;"],
['Joshua','Jsh','10','002','2',"That they feared greatly, because Gibeon \emph{was} a great city, as one of the royal cities, and because it \emph{was} greater than Ai, and all the men thereof \emph{were} mighty."],
['Joshua','Jsh','10','003','3',"Wherefore Adoni-zedek king of Jerusalem sent unto Hoham king of Hebron, and unto Piram king of Jarmuth, and unto Japhia king of Lachish, and unto Debir king of Eglon, saying,"],
['Joshua','Jsh','10','004','4',"Come up unto me, and help me, that we may smite Gibeon: for it hath made peace with Joshua and with the children of Israel."],
['Joshua','Jsh','10','005','5',"Therefore the five kings of the Amorites, the king of Jerusalem, the king of Hebron, the king of Jarmuth, the king of Lachish, the king of Eglon, gathered themselves together, and went up, they and all their hosts, and encamped before Gibeon, and made war against it."],
['Joshua','Jsh','10','006','6',"And the men of Gibeon sent unto Joshua to the camp to Gilgal, saying, Slack not thy hand from thy servants; come up to us quickly, and save us, and help us: for all the kings of the Amorites that dwell in the mountains are gathered together against us."],
['Joshua','Jsh','10','007','7',"So Joshua ascended from Gilgal, he, and all the people of war with him, and all the mighty men of valour."],
['Joshua','Jsh','10','008','8',"And the LORD said unto Joshua, Fear them not: for I have delivered them into thine hand; there shall not a man of them stand before thee."],
['Joshua','Jsh','10','009','9',"Joshua therefore came unto them suddenly, \emph{and} went up from Gilgal all night."],
['Joshua','Jsh','10','010','10',"And the LORD discomfited them before Israel, and slew them with a great slaughter at Gibeon, and chased them along the way that goeth up to Beth-horon, and smote them to Azekah, and unto Makkedah."],
['Joshua','Jsh','10','011','11',"And it came to pass, as they fled from before Israel, \emph{and} were in the going down to Beth-horon, that the LORD cast down great stones from heaven upon them unto Azekah, and they died: \emph{they} \emph{were} more which died with hailstones than \emph{they} whom the children of Israel slew with the sword."],
['Joshua','Jsh','10','012','12',"Then spake Joshua to the LORD in the day when the LORD delivered up the Amorites before the children of Israel, and he said in the sight of Israel, Sun, stand thou still upon Gibeon; and thou, Moon, in the valley of Ajalon."],
['Joshua','Jsh','10','013','13',"And the sun stood still, and the moon stayed, until the people had avenged themselves upon their enemies. \emph{Is} not this written in the book of Jasher? So the sun stood still in the midst of heaven, and hasted not to go down about a whole day."],
['Joshua','Jsh','10','014','14',"And there was no day like that before it or after it, that the LORD hearkened unto the voice of a man: for the LORD fought for Israel."],
['Joshua','Jsh','10','015','15',"And Joshua returned, and all Israel with him, unto the camp to Gilgal."],
['Joshua','Jsh','10','016','16',"But these five kings fled, and hid themselves in a cave at Makkedah."],
['Joshua','Jsh','10','017','17',"And it was told Joshua, saying, The five kings are found hid in a cave at Makkedah."],
['Joshua','Jsh','10','018','18',"And Joshua said, Roll great stones upon the mouth of the cave, and set men by it for to keep them:"],
['Joshua','Jsh','10','019','19',"And stay ye not, \emph{but} pursue after your enemies, and smite the hindmost of them; suffer them not to enter into their cities: for the LORD your God hath delivered them into your hand."],
['Joshua','Jsh','10','020','20',"And it came to pass, when Joshua and the children of Israel had made an end of slaying them with a very great slaughter, till they were consumed, that the rest \emph{which} remained of them entered into fenced cities."],
['Joshua','Jsh','10','021','21',"And all the people returned to the camp to Joshua at Makkedah in peace: none moved his tongue against any of the children of Israel."],
['Joshua','Jsh','10','022','22',"Then said Joshua, Open the mouth of the cave, and bring out those five kings unto me out of the cave."],
['Joshua','Jsh','10','023','23',"And they did so, and brought forth those five kings unto him out of the cave, the king of Jerusalem, the king of Hebron, the king of Jarmuth, the king of Lachish, \emph{and} the king of Eglon."],
['Joshua','Jsh','10','024','24',"And it came to pass, when they brought out those kings unto Joshua, that Joshua called for all the men of Israel, and said unto the captains of the men of war which went with him, Come near, put your feet upon the necks of these kings. And they came near, and put their feet upon the necks of them."],
['Joshua','Jsh','10','025','25',"And Joshua said unto them, Fear not, nor be dismayed, be strong and of good courage: for thus shall the LORD do to all your enemies against whom ye fight."],
['Joshua','Jsh','10','026','26',"And afterward Joshua smote them, and slew them, and hanged them on five trees: and they were hanging upon the trees until the evening."],
['Joshua','Jsh','10','027','27',"And it came to pass at the time of the going down of the sun, \emph{that} Joshua commanded, and they took them down off the trees, and cast them into the cave wherein they had been hid, and laid great stones in the cave's mouth, \emph{which} \emph{remain} until this very day."],
['Joshua','Jsh','10','028','28',"And that day Joshua took Makkedah, and smote it with the edge of the sword, and the king thereof he utterly destroyed, them, and all the souls that \emph{were} therein; he let none remain: and he did to the king of Makkedah as he did unto the king of Jericho."],
['Joshua','Jsh','10','029','29',"Then Joshua passed from Makkedah, and all Israel with him, unto Libnah, and fought against Libnah:"],
['Joshua','Jsh','10','030','30',"And the LORD delivered it also, and the king thereof, into the hand of Israel; and he smote it with the edge of the sword, and all the souls that \emph{were} therein; he let none remain in it; but did unto the king thereof as he did unto the king of Jericho."],
['Joshua','Jsh','10','031','31',"And Joshua passed from Libnah, and all Israel with him, unto Lachish, and encamped against it, and fought against it:"],
['Joshua','Jsh','10','032','32',"And the LORD delivered Lachish into the hand of Israel, which took it on the second day, and smote it with the edge of the sword, and all the souls that \emph{were} therein, according to all that he had done to Libnah."],
['Joshua','Jsh','10','033','33',"Then Horam king of Gezer came up to help Lachish; and Joshua smote him and his people, until he had left him none remaining."],
['Joshua','Jsh','10','034','34',"And from Lachish Joshua passed unto Eglon, and all Israel with him; and they encamped against it, and fought against it:"],
['Joshua','Jsh','10','035','35',"And they took it on that day, and smote it with the edge of the sword, and all the souls that \emph{were} therein he utterly destroyed that day, according to all that he had done to Lachish."],
['Joshua','Jsh','10','036','36',"And Joshua went up from Eglon, and all Israel with him, unto Hebron; and they fought against it:"],
['Joshua','Jsh','10','037','37',"And they took it, and smote it with the edge of the sword, and the king thereof, and all the cities thereof, and all the souls that \emph{were} therein; he left none remaining, according to all that he had done to Eglon; but destroyed it utterly, and all the souls that \emph{were} therein."],
['Joshua','Jsh','10','038','38',"And Joshua returned, and all Israel with him, to Debir; and fought against it:"],
['Joshua','Jsh','10','039','39',"And he took it, and the king thereof, and all the cities thereof; and they smote them with the edge of the sword, and utterly destroyed all the souls that \emph{were} therein; he left none remaining: as he had done to Hebron, so he did to Debir, and to the king thereof; as he had done also to Libnah, and to her king."],
['Joshua','Jsh','10','040','40',"So Joshua smote all the country of the hills, and of the south, and of the vale, and of the springs, and all their kings: he left none remaining, but utterly destroyed all that breathed, as the LORD God of Israel commanded."],
['Joshua','Jsh','10','041','41',"And Joshua smote them from Kadesh-barnea even unto Gaza, and all the country of Goshen, even unto Gibeon."],
['Joshua','Jsh','10','042','42',"And all these kings and their land did Joshua take at one time, because the LORD God of Israel fought for Israel."],
['Joshua','Jsh','10','043','43',"And Joshua returned, and all Israel with him, unto the camp to Gilgal."]
]

Joshua011 = [
    ['Joshua','Jsh','11','001','1',"And it came to pass, when Jabin king of Hazor had heard \emph{those} \emph{things}, that he sent to Jobab king of Madon, and to the king of Shimron, and to the king of Achshaph,"],
['Joshua','Jsh','11','002','2',"And to the kings that \emph{were} on the north of the mountains, and of the plains south of Chinneroth, and in the valley, and in the borders of Dor on the west,"],
['Joshua','Jsh','11','003','3',"\emph{And} \emph{to} the Canaanite on the east and on the west, and \emph{to} the Amorite, and the Hittite, and the Perizzite, and the Jebusite in the mountains, and \emph{to} the Hivite under Hermon in the land of Mizpeh."],
['Joshua','Jsh','11','004','4',"And they went out, they and all their hosts with them, much people, even as the sand that \emph{is} upon the sea shore in multitude, with horses and chariots very many."],
['Joshua','Jsh','11','005','5',"And when all these kings were met together, they came and pitched together at the waters of Merom, to fight against Israel."],
['Joshua','Jsh','11','006','6',"And the LORD said unto Joshua, Be not afraid because of them: for to morrow about this time will I deliver them up all slain before Israel: thou shalt hough their horses, and burn their chariots with fire."],
['Joshua','Jsh','11','007','7',"So Joshua came, and all the people of war with him, against them by the waters of Merom suddenly; and they fell upon them."],
['Joshua','Jsh','11','008','8',"And the LORD delivered them into the hand of Israel, who smote them, and chased them unto great Zidon, and unto Misrephoth-maim, and unto the valley of Mizpeh eastward; and they smote them, until they left them none remaining."],
['Joshua','Jsh','11','009','9',"And Joshua did unto them as the LORD bade him: he houghed their horses, and burnt their chariots with fire."],
['Joshua','Jsh','11','010','10',"And Joshua at that time turned back, and took Hazor, and smote the king thereof with the sword: for Hazor beforetime was the head of all those kingdoms."],
['Joshua','Jsh','11','011','11',"And they smote all the souls that \emph{were} therein with the edge of the sword, utterly destroying \emph{them}: there was not any left to breathe: and he burnt Hazor with fire."],
['Joshua','Jsh','11','012','12',"And all the cities of those kings, and all the kings of them, did Joshua take, and smote them with the edge of the sword, \emph{and} he utterly destroyed them, as Moses the servant of the LORD commanded."],
['Joshua','Jsh','11','013','13',"But \emph{as} \emph{for} the cities that stood still in their strength, Israel burned none of them, save Hazor only; \emph{that} did Joshua burn."],
['Joshua','Jsh','11','014','14',"And all the spoil of these cities, and the cattle, the children of Israel took for a prey unto themselves; but every man they smote with the edge of the sword, until they had destroyed them, neither left they any to breathe."],
['Joshua','Jsh','11','015','15',"As the LORD commanded Moses his servant, so did Moses command Joshua, and so did Joshua; he left nothing undone of all that the LORD commanded Moses."],
['Joshua','Jsh','11','016','16',"So Joshua took all that land, the hills, and all the south country, and all the land of Goshen, and the valley, and the plain, and the mountain of Israel, and the valley of the same;"],
['Joshua','Jsh','11','017','17',"\emph{Even} from the mount Halak, that goeth up to Seir, even unto Baal-gad in the valley of Lebanon under mount Hermon: and all their kings he took, and smote them, and slew them."],
['Joshua','Jsh','11','018','18',"Joshua made war a long time with all those kings."],
['Joshua','Jsh','11','019','19',"There was not a city that made peace with the children of Israel, save the Hivites the inhabitants of Gibeon: all \emph{other} they took in battle."],
['Joshua','Jsh','11','020','20',"For it was of the LORD to harden their hearts, that they should come against Israel in battle, that he might destroy them utterly, \emph{and} that they might have no favour, but that he might destroy them, as the LORD commanded Moses."],
['Joshua','Jsh','11','021','21',"And at that time came Joshua, and cut off the Anakims from the mountains, from Hebron, from Debir, from Anab, and from all the mountains of Judah, and from all the mountains of Israel: Joshua destroyed them utterly with their cities."],
['Joshua','Jsh','11','022','22',"There was none of the Anakims left in the land of the children of Israel: only in Gaza, in Gath, and in Ashdod, there remained."],
['Joshua','Jsh','11','023','23',"So Joshua took the whole land, according to all that the LORD said unto Moses; and Joshua gave it for an inheritance unto Israel according to their divisions by their tribes. And the land rested from war."],
]

Joshua012 = [
    ['Joshua','Jsh','12','001','1',"Now these \emph{are} the kings of the land, which the children of Israel smote, and possessed their land on the other side Jordan toward the rising of the sun, from the river Arnon unto mount Hermon, and all the plain on the east:"],
['Joshua','Jsh','12','002','2',"Sihon king of the Amorites, who dwelt in Heshbon, \emph{and} ruled from Aroer, which \emph{is} upon the bank of the river Arnon, and from the middle of the river, and from half Gilead, even unto the river Jabbok, \emph{which} \emph{is} the border of the children of Ammon;"],
['Joshua','Jsh','12','003','3',"And from the plain to the sea of Chinneroth on the east, and unto the sea of the plain, \emph{even} the salt sea on the east, the way to Beth-jeshimoth; and from the south, under Ashdoth-pisgah:"],
['Joshua','Jsh','12','004','4',"And the coast of Og king of Bashan, \emph{which} \emph{was} of the remnant of the giants, that dwelt at Ashtaroth and at Edrei,"],
['Joshua','Jsh','12','005','5',"And reigned in mount Hermon, and in Salcah, and in all Bashan, unto the border of the Geshurites and the Maachathites, and half Gilead, the border of Sihon king of Heshbon."],
['Joshua','Jsh','12','006','6',"Them did Moses the servant of the LORD and the children of Israel smite: and Moses the servant of the LORD gave it \emph{for} a possession unto the Reubenites, and the Gadites, and the half tribe of Manasseh."],
['Joshua','Jsh','12','007','7',"And these \emph{are} the kings of the country which Joshua and the children of Israel smote on this side Jordan on the west, from Baal-gad in the valley of Lebanon even unto the mount Halak, that goeth up to Seir; which Joshua gave unto the tribes of Israel \emph{for} a possession according to their divisions;"],
['Joshua','Jsh','12','008','8',"In the mountains, and in the valleys, and in the plains, and in the springs, and in the wilderness, and in the south country; the Hittites, the Amorites, and the Canaanites, the Perizzites, the Hivites, and the Jebusites:"],
['Joshua','Jsh','12','009','9',"The king of Jericho, one; the king of Ai, which \emph{is} beside Beth-el, one;"],
['Joshua','Jsh','12','010','10',"The king of Jerusalem, one; the king of Hebron, one;"],
['Joshua','Jsh','12','011','11',"The king of Jarmuth, one; the king of Lachish, one;"],
['Joshua','Jsh','12','012','12',"The king of Eglon, one; the king of Gezer, one;"],
['Joshua','Jsh','12','013','13',"The king of Debir, one; the king of Geder, one;"],
['Joshua','Jsh','12','014','14',"The king of Hormah, one; the king of Arad, one;"],
['Joshua','Jsh','12','015','15',"The king of Libnah, one; the king of Adullam, one;"],
['Joshua','Jsh','12','016','16',"The king of Makkedah, one; the king of Beth-el, one;"],
['Joshua','Jsh','12','017','17',"The king of Tappuah, one; the king of Hepher, one;"],
['Joshua','Jsh','12','018','18',"The king of Aphek, one; the king of Lasharon, one;"],
['Joshua','Jsh','12','019','19',"The king of Madon, one; the king of Hazor, one;"],
['Joshua','Jsh','12','020','20',"The king of Shimron-meron, one; the king of Achshaph, one;"],
['Joshua','Jsh','12','021','21',"The king of Taanach, one; the king of Megiddo, one;"],
['Joshua','Jsh','12','022','22',"The king of Kedesh, one; the king of Jokneam of Carmel, one;"],
['Joshua','Jsh','12','023','23',"The king of Dor in the coast of Dor, one; the king of the nations of Gilgal, one;"],
['Joshua','Jsh','12','024','24',"The king of Tirzah, one: all the kings thirty and one."]
]

Joshua013 = [
    ['Joshua','Jsh','13','001','1',"Now Joshua was old \emph{and} stricken in years; and the LORD said unto him, Thou art old \emph{and} stricken in years, and there remaineth yet very much land to be possessed."],
['Joshua','Jsh','13','002','2',"This \emph{is} the land that yet remaineth: all the borders of the Philistines, and all Geshuri,"],
['Joshua','Jsh','13','003','3',"From Sihor, which \emph{is} before Egypt, even unto the borders of Ekron northward, \emph{which} is counted to the Canaanite: five lords of the Philistines; the Gazathites, and the Ashdothites, the Eshkalonites, the Gittites, and the Ekronites; also the Avites:"],
['Joshua','Jsh','13','004','4',"From the south, all the land of the Canaanites, and Mearah that \emph{is} beside the Sidonians, unto Aphek, to the borders of the Amorites:"],
['Joshua','Jsh','13','005','5',"And the land of the Giblites, and all Lebanon, toward the sunrising, from Baal-gad under mount Hermon unto the entering into Hamath."],
['Joshua','Jsh','13','006','6',"All the inhabitants of the hill country from Lebanon unto Misrephoth-maim, \emph{and} all the Sidonians, them will I drive out from before the children of Israel: only divide thou it by lot unto the Israelites for an inheritance, as I have commanded thee."],
['Joshua','Jsh','13','007','7',"Now therefore divide this land for an inheritance unto the nine tribes, and the half tribe of Manasseh,"],
['Joshua','Jsh','13','008','8',"With whom the Reubenites and the Gadites have received their inheritance, which Moses gave them, beyond Jordan eastward, \emph{even} as Moses the servant of the LORD gave them;"],
['Joshua','Jsh','13','009','9',"From Aroer, that \emph{is} upon the bank of the river Arnon, and the city that \emph{is} in the midst of the river, and all the plain of Medeba unto Dibon;"],
['Joshua','Jsh','13','010','10',"And all the cities of Sihon king of the Amorites, which reigned in Heshbon, unto the border of the children of Ammon;"],
['Joshua','Jsh','13','011','11',"And Gilead, and the border of the Geshurites and Maachathites, and all mount Hermon, and all Bashan unto Salcah;"],
['Joshua','Jsh','13','012','12',"All the kingdom of Og in Bashan, which reigned in Ashtaroth and in Edrei, who remained of the remnant of the giants: for these did Moses smite, and cast them out."],
['Joshua','Jsh','13','013','13',"Nevertheless the children of Israel expelled not the Geshurites, nor the Maachathites: but the Geshurites and the Maachathites dwell among the Israelites until this day."],
['Joshua','Jsh','13','014','14',"Only unto the tribe of Levi he gave none inheritance; the sacrifices of the LORD God of Israel made by fire \emph{are} their inheritance, as he said unto them."],
['Joshua','Jsh','13','015','15',"And Moses gave unto the tribe of the children of Reuben \emph{inheritance} according to their families."],
['Joshua','Jsh','13','016','16',"And their coast was from Aroer, that \emph{is} on the bank of the river Arnon, and the city that \emph{is} in the midst of the river, and all the plain by Medeba;"],
['Joshua','Jsh','13','017','17',"Heshbon, and all her cities that \emph{are} in the plain; Dibon, and Bamoth-baal, and Beth-baal-meon,"],
['Joshua','Jsh','13','018','18',"And Jahazah, and Kedemoth, and Mephaath,"],
['Joshua','Jsh','13','019','19',"And Kirjathaim, and Sibmah, and Zareth-shahar in the mount of the valley,"],
['Joshua','Jsh','13','020','20',"And Beth-peor, and Ashdoth-pisgah, and Beth-jeshimoth,"],
['Joshua','Jsh','13','021','21',"And all the cities of the plain, and all the kingdom of Sihon king of the Amorites, which reigned in Heshbon, whom Moses smote with the princes of Midian, Evi, and Rekem, and Zur, and Hur, and Reba, \emph{which} \emph{were} dukes of Sihon, dwelling in the country."],
['Joshua','Jsh','13','022','22',"Balaam also the son of Beor, the soothsayer, did the children of Israel slay with the sword among them that were slain by them."],
['Joshua','Jsh','13','023','23',"And the border of the children of Reuben was Jordan, and the border \emph{thereof}. This \emph{was} the inheritance of the children of Reuben after their families, the cities and the villages thereof."],
['Joshua','Jsh','13','024','24',"And Moses gave \emph{inheritance} unto the tribe of Gad, \emph{even} unto the children of Gad according to their families."],
['Joshua','Jsh','13','025','25',"And their coast was Jazer, and all the cities of Gilead, and half the land of the children of Ammon, unto Aroer that \emph{is} before Rabbah;"],
['Joshua','Jsh','13','026','26',"And from Heshbon unto Ramath-mizpeh, and Betonim; and from Mahanaim unto the border of Debir;"],
['Joshua','Jsh','13','027','27',"And in the valley, Beth-aram, and Beth-nimrah, and Succoth, and Zaphon, the rest of the kingdom of Sihon king of Heshbon, Jordan and \emph{his} border, \emph{even} unto the edge of the sea of Chinnereth on the other side Jordan eastward."],
['Joshua','Jsh','13','028','28',"This \emph{is} the inheritance of the children of Gad after their families, the cities, and their villages."],
['Joshua','Jsh','13','029','29',"And Moses gave \emph{inheritance} unto the half tribe of Manasseh: and \emph{this} was \emph{the} \emph{possession} of the half tribe of the children of Manasseh by their families."],
['Joshua','Jsh','13','030','30',"And their coast was from Mahanaim, all Bashan, all the kingdom of Og king of Bashan, and all the towns of Jair, which \emph{are} in Bashan, threescore cities:"],
['Joshua','Jsh','13','031','31',"And half Gilead, and Ashtaroth, and Edrei, cities of the kingdom of Og in Bashan, \emph{were} \emph{pertaining} unto the children of Machir the son of Manasseh, \emph{even} to the one half of the children of Machir by their families."],
['Joshua','Jsh','13','032','32',"These \emph{are} \emph{the} \emph{countries} which Moses did distribute for inheritance in the plains of Moab, on the other side Jordan, by Jericho, eastward."],
['Joshua','Jsh','13','033','33',"But unto the tribe of Levi Moses gave not \emph{any} inheritance: the LORD God of Israel \emph{was} their inheritance, as he said unto them."]
]

Joshua014 = [
    ['Joshua','Jsh','14','001','1',"And these \emph{are} \emph{the} \emph{countries} which the children of Israel inherited in the land of Canaan, which Eleazar the priest, and Joshua the son of Nun, and the heads of the fathers of the tribes of the children of Israel, distributed for inheritance to them."],
['Joshua','Jsh','14','002','2',"By lot \emph{was} their inheritance, as the LORD commanded by the hand of Moses, for the nine tribes, and \emph{for} the half tribe."],
['Joshua','Jsh','14','003','3',"For Moses had given the inheritance of two tribes and an half tribe on the other side Jordan: but unto the Levites he gave none inheritance among them."],
['Joshua','Jsh','14','004','4',"For the children of Joseph were two tribes, Manasseh and Ephraim: therefore they gave no part unto the Levites in the land, save cities to dwell \emph{in}, with their suburbs for their cattle and for their substance."],
['Joshua','Jsh','14','005','5',"As the LORD commanded Moses, so the children of Israel did, and they divided the land."],
['Joshua','Jsh','14','006','6',"Then the children of Judah came unto Joshua in Gilgal: and Caleb the son of Jephunneh the Kenezite said unto him, Thou knowest the thing that the LORD said unto Moses the man of God concerning me and thee in Kadesh-barnea."],
['Joshua','Jsh','14','007','7',"Forty years old \emph{was} I when Moses the servant of the LORD sent me from Kadesh-barnea to espy out the land; and I brought him word again as \emph{it} \emph{was} in mine heart."],
['Joshua','Jsh','14','008','8',"Nevertheless my brethren that went up with me made the heart of the people melt: but I wholly followed the LORD my God."],
['Joshua','Jsh','14','009','9',"And Moses sware on that day, saying, Surely the land whereon thy feet have trodden shall be thine inheritance, and thy children's for ever, because thou hast wholly followed the LORD my God."],
['Joshua','Jsh','14','010','10',"And now, behold, the LORD hath kept me alive, as he said, these forty and five years, even since the LORD spake this word unto Moses, while \emph{the} \emph{children} \emph{of} Israel wandered in the wilderness: and now, lo, I \emph{am} this day fourscore and five years old."],
['Joshua','Jsh','14','011','11',"As yet I \emph{am} \emph{as} strong this day as \emph{I} \emph{was} in the day that Moses sent me: as my strength \emph{was} then, even so \emph{is} my strength now, for war, both to go out, and to come in."],
['Joshua','Jsh','14','012','12',"Now therefore give me this mountain, whereof the LORD spake in that day; for thou heardest in that day how the Anakims \emph{were} there, and \emph{that} the cities \emph{were} great \emph{and} fenced: if so be the LORD \emph{will} \emph{be} with me, then I shall be able to drive them out, as the LORD said."],
['Joshua','Jsh','14','013','13',"And Joshua blessed him, and gave unto Caleb the son of Jephunneh Hebron for an inheritance."],
['Joshua','Jsh','14','014','14',"Hebron therefore became the inheritance of Caleb the son of Jephunneh the Kenezite unto this day, because that he wholly followed the LORD God of Israel."],
['Joshua','Jsh','14','015','15',"And the name of Hebron before \emph{was} Kirjath-arba; \emph{which} \emph{Arba} \emph{was} a great man among the Anakims. And the land had rest from war."]
]

Joshua015 = [
    ['Joshua','Jsh','15','001','1',"\emph{This} then was the lot of the tribe of the children of Judah by their families; \emph{even} to the border of Edom the wilderness of Zin southward \emph{was} the uttermost part of the south coast."],
['Joshua','Jsh','15','002','2',"And their south border was from the shore of the salt sea, from the bay that looketh southward:"],
['Joshua','Jsh','15','003','3',"And it went out to the south side to Maaleh-acrabbim, and passed along to Zin, and ascended up on the south side unto Kadesh-barnea, and passed along to Hezron, and went up to Adar, and fetched a compass to Karkaa:"],
['Joshua','Jsh','15','004','4',"\emph{From} \emph{thence} it passed toward Azmon, and went out unto the river of Egypt; and the goings out of that coast were at the sea: this shall be your south coast."],
['Joshua','Jsh','15','005','5',"And the east border \emph{was} the salt sea, \emph{even} unto the end of Jordan. And \emph{their} border in the north quarter \emph{was} from the bay of the sea at the uttermost part of Jordan:"],
['Joshua','Jsh','15','006','6',"And the border went up to Beth-hogla, and passed along by the north of Beth-arabah; and the border went up to the stone of Bohan the son of Reuben:"],
['Joshua','Jsh','15','007','7',"And the border went up toward Debir from the valley of Achor, and so northward, looking toward Gilgal, that \emph{is} before the going up to Adummim, which \emph{is} on the south side of the river: and the border passed toward the waters of En-shemesh, and the goings out thereof were at En-rogel:"],
['Joshua','Jsh','15','008','8',"And the border went up by the valley of the son of Hinnom unto the south side of the Jebusite; the same \emph{is} Jerusalem: and the border went up to the top of the mountain that \emph{lieth} before the valley of Hinnom westward, which \emph{is} at the end of the valley of the giants northward:"],
['Joshua','Jsh','15','009','9',"And the border was drawn from the top of the hill unto the fountain of the water of Nephtoah, and went out to the cities of mount Ephron; and the border was drawn to Baalah, which \emph{is} Kirjath-jearim:"],
['Joshua','Jsh','15','010','10',"And the border compassed from Baalah westward unto mount Seir, and passed along unto the side of mount Jearim, which \emph{is} Chesalon, on the north side, and went down to Beth-shemesh, and passed on to Timnah:"],
['Joshua','Jsh','15','011','11',"And the border went out unto the side of Ekron northward: and the border was drawn to Shicron, and passed along to mount Baalah, and went out unto Jabneel; and the goings out of the border were at the sea."],
['Joshua','Jsh','15','012','12',"And the west border \emph{was} to the great sea, and the coast \emph{thereof}. This \emph{is} the coast of the children of Judah round about according to their families."],
['Joshua','Jsh','15','013','13',"And unto Caleb the son of Jephunneh he gave a part among the children of Judah, according to the commandment of the LORD to Joshua, \emph{even} the city of Arba the father of Anak, which \emph{city} \emph{is} Hebron."],
['Joshua','Jsh','15','014','14',"And Caleb drove thence the three sons of Anak, Sheshai, and Ahiman, and Talmai, the children of Anak."],
['Joshua','Jsh','15','015','15',"And he went up thence to the inhabitants of Debir: and the name of Debir before \emph{was} Kirjath-sepher."],
['Joshua','Jsh','15','016','16',"And Caleb said, He that smiteth Kirjath-sepher, and taketh it, to him will I give Achsah my daughter to wife."],
['Joshua','Jsh','15','017','17',"And Othniel the son of Kenaz, the brother of Caleb, took it: and he gave him Achsah his daughter to wife."],
['Joshua','Jsh','15','018','18',"And it came to pass, as she came \emph{unto} \emph{him}, that she moved him to ask of her father a field: and she lighted off \emph{her} ass; and Caleb said unto her, What wouldest thou?"],
['Joshua','Jsh','15','019','19',"Who answered, Give me a blessing; for thou hast given me a south land; give me also springs of water. And he gave her the upper springs, and the nether springs."],
['Joshua','Jsh','15','020','20',"This \emph{is} the inheritance of the tribe of the children of Judah according to their families."],
['Joshua','Jsh','15','021','21',"And the uttermost cities of the tribe of the children of Judah toward the coast of Edom southward were Kabzeel, and Eder, and Jagur,"],
['Joshua','Jsh','15','022','22',"And Kinah, and Dimonah, and Adadah,"],
['Joshua','Jsh','15','023','23',"And Kedesh, and Hazor, and Ithnan,"],
['Joshua','Jsh','15','024','24',"Ziph, and Telem, and Bealoth,"],
['Joshua','Jsh','15','025','25',"And Hazor, Hadattah, and Kerioth, \emph{and} Hezron, which \emph{is} Hazor,"],
['Joshua','Jsh','15','026','26',"Amam, and Shema, and Moladah,"],
['Joshua','Jsh','15','027','27',"And Hazar-gaddah, and Heshmon, and Beth-palet,"],
['Joshua','Jsh','15','028','28',"And Hazar-shual, and Beer-sheba, and Bizjothjah,"],
['Joshua','Jsh','15','029','29',"Baalah, and Iim, and Azem,"],
['Joshua','Jsh','15','030','30',"And Eltolad, and Chesil, and Hormah,"],
['Joshua','Jsh','15','031','31',"And Ziklag, and Madmannah, and Sansannah,"],
['Joshua','Jsh','15','032','32',"And Lebaoth, and Shilhim, and Ain, and Rimmon: all the cities \emph{are} twenty and nine, with their villages:"],
['Joshua','Jsh','15','033','33',"\emph{And} in the valley, Eshtaol, and Zoreah, and Ashnah,"],
['Joshua','Jsh','15','034','34',"And Zanoah, and En-gannim, Tappuah, and Enam,"],
['Joshua','Jsh','15','035','35',"Jarmuth, and Adullam, Socoh, and Azekah,"],
['Joshua','Jsh','15','036','36',"And Sharaim, and Adithaim, and Gederah, and Gederothaim; fourteen cities with their villages:"],
['Joshua','Jsh','15','037','37',"Zenan, and Hadashah, and Migdal-gad,"],
['Joshua','Jsh','15','038','38',"And Dilean, and Mizpeh, and Joktheel,"],
['Joshua','Jsh','15','039','39',"Lachish, and Bozkath, and Eglon,"],
['Joshua','Jsh','15','040','40',"And Cabbon, and Lahmam, and Kithlish,"],
['Joshua','Jsh','15','041','41',"And Gederoth, Beth-dagon, and Naamah, and Makkedah; sixteen cities with their villages:"],
['Joshua','Jsh','15','042','42',"Libnah, and Ether, and Ashan,"],
['Joshua','Jsh','15','043','43',"And Jiphtah, and Ashnah, and Nezib,"],
['Joshua','Jsh','15','044','44',"And Keilah, and Achzib, and Mareshah; nine cities with their villages:"],
['Joshua','Jsh','15','045','45',"Ekron, with her towns and her villages:"],
['Joshua','Jsh','15','046','46',"From Ekron even unto the sea, all that \emph{lay} near Ashdod, with their villages:"],
['Joshua','Jsh','15','047','47',"Ashdod with her towns and her villages, Gaza with her towns and her villages, unto the river of Egypt, and the great sea, and the border \emph{thereof}:"],
['Joshua','Jsh','15','048','48',"And in the mountains, Shamir, and Jattir, and Socoh,"],
['Joshua','Jsh','15','049','49',"And Dannah, and Kirjath-sannah, which \emph{is} Debir,"],
['Joshua','Jsh','15','050','50',"And Anab, and Eshtemoh, and Anim,"],
['Joshua','Jsh','15','051','51',"And Goshen, and Holon, and Giloh; eleven cities with their villages:"],
['Joshua','Jsh','15','052','52',"Arab, and Dumah, and Eshean,"],
['Joshua','Jsh','15','053','53',"And Janum, and Beth-tappuah, and Aphekah,"],
['Joshua','Jsh','15','054','54',"And Humtah, and Kirjath-arba, which \emph{is} Hebron, and Zior; nine cities with their villages:"],
['Joshua','Jsh','15','055','55',"Maon, Carmel, and Ziph, and Juttah,"],
['Joshua','Jsh','15','056','56',"And Jezreel, and Jokdeam, and Zanoah,"],
['Joshua','Jsh','15','057','57',"Cain, Gibeah, and Timnah; ten cities with their villages:"],
['Joshua','Jsh','15','058','58',"Halhul, Beth-zur, and Gedor,"],
['Joshua','Jsh','15','059','59',"And Maarath, and Beth-anoth, and Eltekon; six cities with their villages:"],
['Joshua','Jsh','15','060','60',"Kirjath-baal, which \emph{is} Kirjath-jearim, and Rabbah; two cities with their villages:"],
['Joshua','Jsh','15','061','61',"In the wilderness, Beth-arabah, Middin, and Secacah,"],
['Joshua','Jsh','15','062','62',"And Nibshan, and the city of Salt, and En-gedi; six cities with their villages."],
['Joshua','Jsh','15','063','63',"As for the Jebusites the inhabitants of Jerusalem, the children of Judah could not drive them out: but the Jebusites dwell with the children of Judah at Jerusalem unto this day."]
]

Joshua016 = [
    ['Joshua','Jsh','16','001','1',"And the lot of the children of Joseph fell from Jordan by Jericho, unto the water of Jericho on the east, to the wilderness that goeth up from Jericho throughout mount Beth-el,"],
['Joshua','Jsh','16','002','2',"And goeth out from Beth-el to Luz, and passeth along unto the borders of Archi to Ataroth,"],
['Joshua','Jsh','16','003','3',"And goeth down westward to the coast of Japhleti, unto the coast of Beth-horon the nether, and to Gezer: and the goings out thereof are at the sea."],
['Joshua','Jsh','16','004','4',"So the children of Joseph, Manasseh and Ephraim, took their inheritance."],
['Joshua','Jsh','16','005','5',"And the border of the children of Ephraim according to their families was \emph{thus}: even the border of their inheritance on the east side was Ataroth-addar, unto Beth-horon the upper;"],
['Joshua','Jsh','16','006','6',"And the border went out toward the sea to Michmethah on the north side; and the border went about eastward unto Taanath-shiloh, and passed by it on the east to Janohah;"],
['Joshua','Jsh','16','007','7',"And it went down from Janohah to Ataroth, and to Naarath, and came to Jericho, and went out at Jordan."],
['Joshua','Jsh','16','008','8',"The border went out from Tappuah westward unto the river Kanah; and the goings out thereof were at the sea. This \emph{is} the inheritance of the tribe of the children of Ephraim by their families."],
['Joshua','Jsh','16','009','9',"And the separate cities for the children of Ephraim \emph{were} among the inheritance of the children of Manasseh, all the cities with their villages."],
['Joshua','Jsh','16','010','10',"And they drave not out the Canaanites that dwelt in Gezer: but the Canaanites dwell among the Ephraimites unto this day, and serve under tribute."]
]

Joshua017 = [
    ['Joshua','Jsh','17','001','1',"There was also a lot for the tribe of Manasseh; for he \emph{was} the firstborn of Joseph; \emph{to} \emph{wit}, for Machir the firstborn of Manasseh, the father of Gilead: because he was a man of war, therefore he had Gilead and Bashan."],
['Joshua','Jsh','17','002','2',"There was also \emph{a} \emph{lot} for the rest of the children of Manasseh by their families; for the children of Abiezer, and for the children of Helek, and for the children of Asriel, and for the children of Shechem, and for the children of Hepher, and for the children of Shemida: these \emph{were} the male children of Manasseh the son of Joseph by their families."],
['Joshua','Jsh','17','003','3',"But Zelophehad, the son of Hepher, the son of Gilead, the son of Machir, the son of Manasseh, had no sons, but daughters: and these \emph{are} the names of his daughters, Mahlah, and Noah, Hoglah, Milcah, and Tirzah."],
['Joshua','Jsh','17','004','4',"And they came near before Eleazar the priest, and before Joshua the son of Nun, and before the princes, saying, The LORD commanded Moses to give us an inheritance among our brethren. Therefore according to the commandment of the LORD he gave them an inheritance among the brethren of their father."],
['Joshua','Jsh','17','005','5',"And there fell ten portions to Manasseh, beside the land of Gilead and Bashan, which \emph{were} on the other side Jordan;"],
['Joshua','Jsh','17','006','6',"Because the daughters of Manasseh had an inheritance among his sons: and the rest of Manasseh's sons had the land of Gilead."],
['Joshua','Jsh','17','007','7',"And the coast of Manasseh was from Asher to Michmethah, that \emph{lieth} before Shechem; and the border went along on the right hand unto the inhabitants of En-tappuah."],
['Joshua','Jsh','17','008','8',"\emph{Now} Manasseh had the land of Tappuah: but Tappuah on the border of Manasseh \emph{belonged} to the children of Ephraim;"],
['Joshua','Jsh','17','009','9',"And the coast descended unto the river Kanah, southward of the river: these cities of Ephraim \emph{are} among the cities of Manasseh: the coast of Manasseh also \emph{was} on the north side of the river, and the outgoings of it were at the sea:"],
['Joshua','Jsh','17','010','10',"Southward \emph{it} \emph{was} Ephraim's, and northward \emph{it} \emph{was} Manasseh's, and the sea is his border; and they met together in Asher on the north, and in Issachar on the east."],
['Joshua','Jsh','17','011','11',"And Manasseh had in Issachar and in Asher Beth-shean and her towns, and Ibleam and her towns, and the inhabitants of Dor and her towns, and the inhabitants of En-dor and her towns, and the inhabitants of Taanach and her towns, and the inhabitants of Megiddo and her towns, \emph{even} three countries."],
['Joshua','Jsh','17','012','12',"Yet the children of Manasseh could not drive out \emph{the} \emph{inhabitants} \emph{of} those cities; but the Canaanites would dwell in that land."],
['Joshua','Jsh','17','013','13',"Yet it came to pass, when the children of Israel were waxen strong, that they put the Canaanites to tribute; but did not utterly drive them out."],
['Joshua','Jsh','17','014','14',"And the children of Joseph spake unto Joshua, saying, Why hast thou given me \emph{but} one lot and one portion to inherit, seeing I \emph{am} a great people, forasmuch as the LORD hath blessed me hitherto?"],
['Joshua','Jsh','17','015','15',"And Joshua answered them, If thou \emph{be} a great people, \emph{then} get thee up to the wood \emph{country}, and cut down for thyself there in the land of the Perizzites and of the giants, if mount Ephraim be too narrow for thee."],
['Joshua','Jsh','17','016','16',"And the children of Joseph said, The hill is not enough for us: and all the Canaanites that dwell in the land of the valley have chariots of iron, \emph{both} \emph{they} who \emph{are} of Beth-shean and her towns, and \emph{they} who \emph{are} of the valley of Jezreel."],
['Joshua','Jsh','17','017','17',"And Joshua spake unto the house of Joseph, \emph{even} to Ephraim and to Manasseh, saying, Thou \emph{art} a great people, and hast great power: thou shalt not have one lot \emph{only}:"],
['Joshua','Jsh','17','018','18',"But the mountain shall be thine; for it \emph{is} a wood, and thou shalt cut it down: and the outgoings of it shall be thine: for thou shalt drive out the Canaanites, though they have iron chariots, \emph{and} though they \emph{be} strong."]
]

Joshua018 = [
    ['Joshua','Jsh','18','001','1',"And the whole congregation of the children of Israel assembled together at Shiloh, and set up the tabernacle of the congregation there. And the land was subdued before them."],
['Joshua','Jsh','18','002','2',"And there remained among the children of Israel seven tribes, which had not yet received their inheritance."],
['Joshua','Jsh','18','003','3',"And Joshua said unto the children of Israel, How long \emph{are} ye slack to go to possess the land, which the LORD God of your fathers hath given you?"],
['Joshua','Jsh','18','004','4',"Give out from among you three men for \emph{each} tribe: and I will send them, and they shall rise, and go through the land, and describe it according to the inheritance of them; and they shall come \emph{again} to me."],
['Joshua','Jsh','18','005','5',"And they shall divide it into seven parts: Judah shall abide in their coast on the south, and the house of Joseph shall abide in their coasts on the north."],
['Joshua','Jsh','18','006','6',"Ye shall therefore describe the land \emph{into} seven parts, and bring \emph{the} \emph{description} hither to me, that I may cast lots for you here before the LORD our God."],
['Joshua','Jsh','18','007','7',"But the Levites have no part among you; for the priesthood of the LORD \emph{is} their inheritance: and Gad, and Reuben, and half the tribe of Manasseh, have received their inheritance beyond Jordan on the east, which Moses the servant of the LORD gave them."],
['Joshua','Jsh','18','008','8',"And the men arose, and went away: and Joshua charged them that went to describe the land, saying, Go and walk through the land, and describe it, and come again to me, that I may here cast lots for you before the LORD in Shiloh."],
['Joshua','Jsh','18','009','9',"And the men went and passed through the land, and described it by cities into seven parts in a book, and came \emph{again} to Joshua to the host at Shiloh."],
['Joshua','Jsh','18','010','10',"And Joshua cast lots for them in Shiloh before the LORD: and there Joshua divided the land unto the children of Israel according to their divisions."],
['Joshua','Jsh','18','011','11',"And the lot of the tribe of the children of Benjamin came up according to their families: and the coast of their lot came forth between the children of Judah and the children of Joseph."],
['Joshua','Jsh','18','012','12',"And their border on the north side was from Jordan; and the border went up to the side of Jericho on the north side, and went up through the mountains westward; and the goings out thereof were at the wilderness of Beth-aven."],
['Joshua','Jsh','18','013','13',"And the border went over from thence toward Luz, to the side of Luz, which \emph{is} Beth-el, southward; and the border descended to Ataroth-adar, near the hill that \emph{lieth} on the south side of the nether Beth-horon."],
['Joshua','Jsh','18','014','14',"And the border was drawn \emph{thence}, and compassed the corner of the sea southward, from the hill that \emph{lieth} before Beth-horon southward; and the goings out thereof were at Kirjath-baal, which \emph{is} Kirjath-jearim, a city of the children of Judah: this \emph{was} the west quarter."],
['Joshua','Jsh','18','015','15',"And the south quarter \emph{was} from the end of Kirjath-jearim, and the border went out on the west, and went out to the well of waters of Nephtoah:"],
['Joshua','Jsh','18','016','16',"And the border came down to the end of the mountain that \emph{lieth} before the valley of the son of Hinnom, \emph{and} which \emph{is} in the valley of the giants on the north, and descended to the valley of Hinnom, to the side of Jebusi on the south, and descended to En-rogel,"],
['Joshua','Jsh','18','017','17',"And was drawn from the north, and went forth to En-shemesh, and went forth toward Geliloth, which \emph{is} over against the going up of Adummim, and descended to the stone of Bohan the son of Reuben,"],
['Joshua','Jsh','18','018','18',"And passed along toward the side over against Arabah northward, and went down unto Arabah:"],
['Joshua','Jsh','18','019','19',"And the border passed along to the side of Beth-hoglah northward: and the outgoings of the border were at the north bay of the salt sea at the south end of Jordan: this \emph{was} the south coast."],
['Joshua','Jsh','18','020','20',"And Jordan was the border of it on the east side. This \emph{was} the inheritance of the children of Benjamin, by the coasts thereof round about, according to their families."],
['Joshua','Jsh','18','021','21',"Now the cities of the tribe of the children of Benjamin according to their families were Jericho, and Beth-hoglah, and the valley of Keziz,"],
['Joshua','Jsh','18','022','22',"And Beth-arabah, and Zemaraim, and Beth-el,"],
['Joshua','Jsh','18','023','23',"And Avim, and Parah, and Ophrah,"],
['Joshua','Jsh','18','024','24',"And Chephar-haammonai, and Ophni, and Gaba; twelve cities with their villages:"],
['Joshua','Jsh','18','025','25',"Gibeon, and Ramah, and Beeroth,"],
['Joshua','Jsh','18','026','26',"And Mizpeh, and Chephirah, and Mozah,"],
['Joshua','Jsh','18','027','27',"And Rekem, and Irpeel, and Taralah,"],
['Joshua','Jsh','18','028','28',"And Zelah, Eleph, and Jebusi, which \emph{is} Jerusalem, Gibeath, \emph{and} Kirjath; fourteen cities with their villages. This \emph{is} the inheritance of the children of Benjamin according to their families."]
]

Joshua019 = [
['Joshua','Jsh','19','001','1',"And the second lot came forth to Simeon, \emph{even} for the tribe of the children of Simeon according to their families: and their inheritance was within the inheritance of the children of Judah."],
['Joshua','Jsh','19','002','2',"And they had in their inheritance Beer-sheba, or Sheba, and Moladah,"],
['Joshua','Jsh','19','003','3',"And Hazar-shual, and Balah, and Azem,"],
['Joshua','Jsh','19','004','4',"And Eltolad, and Bethul, and Hormah,"],
['Joshua','Jsh','19','005','5',"And Ziklag, and Beth-marcaboth, and Hazar-susah,"],
['Joshua','Jsh','19','006','6',"And Beth-lebaoth, and Sharuhen; thirteen cities and their villages:"],
['Joshua','Jsh','19','007','7',"Ain, Remmon, and Ether, and Ashan; four cities and their villages:"],
['Joshua','Jsh','19','008','8',"And all the villages that \emph{were} round about these cities to Baalath-beer, Ramath of the south. This \emph{is} the inheritance of the tribe of the children of Simeon according to their families."],
['Joshua','Jsh','19','009','9',"Out of the portion of the children of Judah \emph{was} the inheritance of the children of Simeon: for the part of the children of Judah was too much for them: therefore the children of Simeon had their inheritance within the inheritance of them."],
['Joshua','Jsh','19','010','10',"And the third lot came up for the children of Zebulun according to their families: and the border of their inheritance was unto Sarid:"],
['Joshua','Jsh','19','011','11',"And their border went up toward the sea, and Maralah, and reached to Dabbasheth, and reached to the river that \emph{is} before Jokneam;"],
['Joshua','Jsh','19','012','12',"And turned from Sarid eastward toward the sunrising unto the border of Chisloth-tabor, and then goeth out to Daberath, and goeth up to Japhia,"],
['Joshua','Jsh','19','013','13',"And from thence passeth on along on the east to Gittah-hepher, to Ittah-kazin, and goeth out to Remmon-methoar to Neah;"],
['Joshua','Jsh','19','014','14',"And the border compasseth it on the north side to Hannathon: and the outgoings thereof are in the valley of Jiphthah-el:"],
['Joshua','Jsh','19','015','15',"And Kattath, and Nahallal, and Shimron, and Idalah, and Beth-lehem: twelve cities with their villages."],
['Joshua','Jsh','19','016','16',"This \emph{is} the inheritance of the children of Zebulun according to their families, these cities with their villages."],
['Joshua','Jsh','19','017','17',"\emph{And} the fourth lot came out to Issachar, for the children of Issachar according to their families."],
['Joshua','Jsh','19','018','18',"And their border was toward Jezreel, and Chesulloth, and Shunem,"],
['Joshua','Jsh','19','019','19',"And Haphraim, and Shion, and Anaharath,"],
['Joshua','Jsh','19','020','20',"And Rabbith, and Kishion, and Abez,"],
['Joshua','Jsh','19','021','21',"And Remeth, and En-gannim, and En-haddah, and Beth-pazzez;"],
['Joshua','Jsh','19','022','22',"And the coast reacheth to Tabor, and Shahazimah, and Beth-shemesh; and the outgoings of their border were at Jordan: sixteen cities with their villages."],
['Joshua','Jsh','19','023','23',"This \emph{is} the inheritance of the tribe of the children of Issachar according to their families, the cities and their villages."],
['Joshua','Jsh','19','024','24',"And the fifth lot came out for the tribe of the children of Asher according to their families."],
['Joshua','Jsh','19','025','25',"And their border was Helkath, and Hali, and Beten, and Achshaph,"],
['Joshua','Jsh','19','026','26',"And Alammelech, and Amad, and Misheal; and reacheth to Carmel westward, and to Shihor-libnath;"],
['Joshua','Jsh','19','027','27',"And turneth toward the sunrising to Beth-dagon, and reacheth to Zebulun, and to the valley of Jiphthah-el toward the north side of Beth-emek, and Neiel, and goeth out to Cabul on the left hand,"],
['Joshua','Jsh','19','028','28',"And Hebron, and Rehob, and Hammon, and Kanah, \emph{even} unto great Zidon;"],
['Joshua','Jsh','19','029','29',"And \emph{then} the coast turneth to Ramah, and to the strong city Tyre; and the coast turneth to Hosah; and the outgoings thereof are at the sea from the coast to Achzib:"],
['Joshua','Jsh','19','030','30',"Ummah also, and Aphek, and Rehob: twenty and two cities with their villages."],
['Joshua','Jsh','19','031','31',"This \emph{is} the inheritance of the tribe of the children of Asher according to their families, these cities with their villages."],
['Joshua','Jsh','19','032','32',"The sixth lot came out to the children of Naphtali, \emph{even} for the children of Naphtali according to their families."],
['Joshua','Jsh','19','033','33',"And their coast was from Heleph, from Allon to Zaanannim, and Adami, Nekeb, and Jabneel, unto Lakum; and the outgoings thereof were at Jordan:"],
['Joshua','Jsh','19','034','34',"And \emph{then} the coast turneth westward to Aznoth-tabor, and goeth out from thence to Hukkok, and reacheth to Zebulun on the south side, and reacheth to Asher on the west side, and to Judah upon Jordan toward the sunrising."],
['Joshua','Jsh','19','035','35',"And the fenced cities \emph{are} Ziddim, Zer, and Hammath, Rakkath, and Chinnereth,"],
['Joshua','Jsh','19','036','36',"And Adamah, and Ramah, and Hazor,"],
['Joshua','Jsh','19','037','37',"And Kedesh, and Edrei, and En-hazor,"],
['Joshua','Jsh','19','038','38',"And Iron, and Migdal-el, Horem, and Beth-anath, and Beth-shemesh; nineteen cities with their villages."],
['Joshua','Jsh','19','039','39',"This \emph{is} the inheritance of the tribe of the children of Naphtali according to their families, the cities and their villages."],
['Joshua','Jsh','19','040','40',"\emph{And} the seventh lot came out for the tribe of the children of Dan according to their families."],
['Joshua','Jsh','19','041','41',"And the coast of their inheritance was Zorah, and Eshtaol, and Ir-shemesh,"],
['Joshua','Jsh','19','042','42',"And Shaalabbin, and Ajalon, and Jethlah,"],
['Joshua','Jsh','19','043','43',"And Elon, and Thimnathah, and Ekron,"],
['Joshua','Jsh','19','044','44',"And Eltekeh, and Gibbethon, and Baalath,"],
['Joshua','Jsh','19','045','45',"And Jehud, and Bene-berak, and Gath-rimmon,"],
['Joshua','Jsh','19','046','46',"And Me-jarkon, and Rakkon, with the border before Japho."],
['Joshua','Jsh','19','047','47',"And the coast of the children of Dan went out \emph{too} \emph{little} for them: therefore the children of Dan went up to fight against Leshem, and took it, and smote it with the edge of the sword, and possessed it, and dwelt therein, and called Leshem, Dan, after the name of Dan their father."],
['Joshua','Jsh','19','048','48',"This \emph{is} the inheritance of the tribe of the children of Dan according to their families, these cities with their villages."],
['Joshua','Jsh','19','049','49',"When they had made an end of dividing the land for inheritance by their coasts, the children of Israel gave an inheritance to Joshua the son of Nun among them:"],
['Joshua','Jsh','19','050','50',"According to the word of the LORD they gave him the city which he asked, \emph{even} Timnath-serah in mount Ephraim: and he built the city, and dwelt therein."],
['Joshua','Jsh','19','051','51',"These \emph{are} the inheritances, which Eleazar the priest, and Joshua the son of Nun, and the heads of the fathers of the tribes of the children of Israel, divided for an inheritance by lot in Shiloh before the LORD, at the door of the tabernacle of the congregation. So they made an end of dividing the country."]]

Joshua020 = [
    ['Joshua','Jsh','20','001','1',"The LORD also spake unto Joshua, saying,"],
    ['Joshua','Jsh','20','002','2',"Speak to the children of Israel, saying, Appoint out for you cities of refuge, whereof I spake unto you by the hand of Moses:"],
    ['Joshua','Jsh','20','003','3',"That the slayer that killeth \emph{any} person unawares \emph{and} unwittingly may flee thither: and they shall be your refuge from the avenger of blood."],
    ['Joshua','Jsh','20','004','4',"And when he that doth flee unto one of those cities shall stand at the entering of the gate of the city, and shall declare his cause in the ears of the elders of that city, they shall take him into the city unto them, and give him a place, that he may dwell among them."],
    ['Joshua','Jsh','20','005','5',"And if the avenger of blood pursue after him, then they shall not deliver the slayer up into his hand; because he smote his neighbour unwittingly, and hated him not beforetime."],
    ['Joshua','Jsh','20','006','6',"And he shall dwell in that city, until he stand before the congregation for judgment, \emph{and} until the death of the high priest that shall be in those days: then shall the slayer return, and come unto his own city, and unto his own house, unto the city from whence he fled."],
    ['Joshua','Jsh','20','007','7',"And they appointed Kedesh in Galilee in mount Naphtali, and Shechem in mount Ephraim, and Kirjath-arba, which \emph{is} Hebron, in the mountain of Judah."],
    ['Joshua','Jsh','20','008','8',"And on the other side Jordan by Jericho eastward, they assigned Bezer in the wilderness upon the plain out of the tribe of Reuben, and Ramoth in Gilead out of the tribe of Gad, and Golan in Bashan out of the tribe of Manasseh."],
    ['Joshua','Jsh','20','009','9',"These were the cities appointed for all the children of Israel, and for the stranger that sojourneth among them, that whosoever killeth \emph{any} person at unawares might flee thither, and not die by the hand of the avenger of blood, until he stood before the congregation."]
]

Joshua021 = [
    ['Joshua','Jsh','21','001','1',"Then came near the heads of the fathers of the Levites unto Eleazar the priest, and unto Joshua the son of Nun, and unto the heads of the fathers of the tribes of the children of Israel;"],
['Joshua','Jsh','21','002','2',"And they spake unto them at Shiloh in the land of Canaan, saying, The LORD commanded by the hand of Moses to give us cities to dwell in, with the suburbs thereof for our cattle."],
['Joshua','Jsh','21','003','3',"And the children of Israel gave unto the Levites out of their inheritance, at the commandment of the LORD, these cities and their suburbs."],
['Joshua','Jsh','21','004','4',"And the lot came out for the families of the Kohathites: and the children of Aaron the priest, \emph{which} \emph{were} of the Levites, had by lot out of the tribe of Judah, and out of the tribe of Simeon, and out of the tribe of Benjamin, thirteen cities."],
['Joshua','Jsh','21','005','5',"And the rest of the children of Kohath \emph{had} by lot out of the families of the tribe of Ephraim, and out of the tribe of Dan, and out of the half tribe of Manasseh, ten cities."],
['Joshua','Jsh','21','006','6',"And the children of Gershon \emph{had} by lot out of the families of the tribe of Issachar, and out of the tribe of Asher, and out of the tribe of Naphtali, and out of the half tribe of Manasseh in Bashan, thirteen cities."],
['Joshua','Jsh','21','007','7',"The children of Merari by their families \emph{had} out of the tribe of Reuben, and out of the tribe of Gad, and out of the tribe of Zebulun, twelve cities."],
['Joshua','Jsh','21','008','8',"And the children of Israel gave by lot unto the Levites these cities with their suburbs, as the LORD commanded by the hand of Moses."],
['Joshua','Jsh','21','009','9',"And they gave out of the tribe of the children of Judah, and out of the tribe of the children of Simeon, these cities which are \emph{here} mentioned by name,"],
['Joshua','Jsh','21','010','10',"Which the children of Aaron, \emph{being} of the families of the Kohathites, \emph{who} \emph{were} of the children of Levi, had: for their's was the first lot."],
['Joshua','Jsh','21','011','11',"And they gave them the city of Arba the father of Anak, which \emph{city} \emph{is} Hebron, in the hill \emph{country} of Judah, with the suburbs thereof round about it."],
['Joshua','Jsh','21','012','12',"But the fields of the city, and the villages thereof, gave they to Caleb the son of Jephunneh for his possession."],
['Joshua','Jsh','21','013','13',"Thus they gave to the children of Aaron the priest Hebron with her suburbs, \emph{to} \emph{be} a city of refuge for the slayer; and Libnah with her suburbs,"],
['Joshua','Jsh','21','014','14',"And Jattir with her suburbs, and Eshtemoa with her suburbs,"],
['Joshua','Jsh','21','015','15',"And Holon with her suburbs, and Debir with her suburbs,"],
['Joshua','Jsh','21','016','16',"And Ain with her suburbs, and Juttah with her suburbs, \emph{and} Beth-shemesh with her suburbs; nine cities out of those two tribes."],
['Joshua','Jsh','21','017','17',"And out of the tribe of Benjamin, Gibeon with her suburbs, Geba with her suburbs,"],
['Joshua','Jsh','21','018','18',"Anathoth with her suburbs, and Almon with her suburbs; four cities."],
['Joshua','Jsh','21','019','19',"All the cities of the children of Aaron, the priests, \emph{were} thirteen cities with their suburbs."],
['Joshua','Jsh','21','020','20',"And the families of the children of Kohath, the Levites which remained of the children of Kohath, even they had the cities of their lot out of the tribe of Ephraim."],
['Joshua','Jsh','21','021','21',"For they gave them Shechem with her suburbs in mount Ephraim, \emph{to} \emph{be} a city of refuge for the slayer; and Gezer with her suburbs,"],
['Joshua','Jsh','21','022','22',"And Kibzaim with her suburbs, and Beth-horon with her suburbs; four cities."],
['Joshua','Jsh','21','023','23',"And out of the tribe of Dan, Eltekeh with her suburbs, Gibbethon with her suburbs,"],
['Joshua','Jsh','21','024','24',"Aijalon with her suburbs, Gath-rimmon with her suburbs; four cities."],
['Joshua','Jsh','21','025','25',"And out of the half tribe of Manasseh, Tanach with her suburbs, and Gath-rimmon with her suburbs; two cities."],
['Joshua','Jsh','21','026','26',"All the cities \emph{were} ten with their suburbs for the families of the children of Kohath that remained."],
['Joshua','Jsh','21','027','27',"And unto the children of Gershon, of the families of the Levites, out of the \emph{other} half tribe of Manasseh \emph{they} \emph{gave} Golan in Bashan with her suburbs, \emph{to} \emph{be} a city of refuge for the slayer; and Beesh-terah with her suburbs; two cities."],
['Joshua','Jsh','21','028','28',"And out of the tribe of Issachar, Kishon with her suburbs, Dabareh with her suburbs,"],
['Joshua','Jsh','21','029','29',"Jarmuth with her suburbs, En-gannim with her suburbs; four cities."],
['Joshua','Jsh','21','030','30',"And out of the tribe of Asher, Mishal with her suburbs, Abdon with her suburbs,"],
['Joshua','Jsh','21','031','31',"Helkath with her suburbs, and Rehob with her suburbs; four cities."],
['Joshua','Jsh','21','032','32',"And out of the tribe of Naphtali, Kedesh in Galilee with her suburbs, \emph{to} \emph{be} a city of refuge for the slayer; and Hammoth-dor with her suburbs, and Kartan with her suburbs; three cities."],
['Joshua','Jsh','21','033','33',"All the cities of the Gershonites according to their families \emph{were} thirteen cities with their suburbs."],
['Joshua','Jsh','21','034','34',"And unto the families of the children of Merari, the rest of the Levites, out of the tribe of Zebulun, Jokneam with her suburbs, and Kartah with her suburbs,"],
['Joshua','Jsh','21','035','35',"Dimnah with her suburbs, Nahalal with her suburbs; four cities."],
['Joshua','Jsh','21','036','36',"And out of the tribe of Reuben, Bezer with her suburbs, and Jahazah with her suburbs,"],
['Joshua','Jsh','21','037','37',"Kedemoth with her suburbs, and Mephaath with her suburbs; four cities."],
['Joshua','Jsh','21','038','38',"And out of the tribe of Gad, Ramoth in Gilead with her suburbs, \emph{to} \emph{be} a city of refuge for the slayer; and Mahanaim with her suburbs,"],
['Joshua','Jsh','21','039','39',"Heshbon with her suburbs, Jazer with her suburbs; four cities in all."],
['Joshua','Jsh','21','040','40',"So all the cities for the children of Merari by their families, which were remaining of the families of the Levites, were \emph{by} their lot twelve cities."],
['Joshua','Jsh','21','041','41',"All the cities of the Levites within the possession of the children of Israel \emph{were} forty and eight cities with their suburbs."],
['Joshua','Jsh','21','042','42',"These cities were every one with their suburbs round about them: thus \emph{were} all these cities."],
['Joshua','Jsh','21','043','43',"And the LORD gave unto Israel all the land which he sware to give unto their fathers; and they possessed it, and dwelt therein."],
['Joshua','Jsh','21','044','44',"And the LORD gave them rest round about, according to all that he sware unto their fathers: and there stood not a man of all their enemies before them; the LORD delivered all their enemies into their hand."],
['Joshua','Jsh','21','045','45',"There failed not ought of any good thing which the LORD had spoken unto the house of Israel; all came to pass."]]

Joshua022 = [
    ['Joshua','Jsh','22','001','1',"Then Joshua called the Reubenites, and the Gadites, and the half tribe of Manasseh,"],
['Joshua','Jsh','22','002','2',"And said unto them, Ye have kept all that Moses the servant of the LORD commanded you, and have obeyed my voice in all that I commanded you:"],
['Joshua','Jsh','22','003','3',"Ye have not left your brethren these many days unto this day, but have kept the charge of the commandment of the LORD your God."],
['Joshua','Jsh','22','004','4',"And now the LORD your God hath given rest unto your brethren, as he promised them: therefore now return ye, and get you unto your tents, \emph{and} unto the land of your possession, which Moses the servant of the LORD gave you on the other side Jordan."],
['Joshua','Jsh','22','005','5',"But take diligent heed to do the commandment and the law, which Moses the servant of the LORD charged you, to love the LORD your God, and to walk in all his ways, and to keep his commandments, and to cleave unto him, and to serve him with all your heart and with all your soul."],
['Joshua','Jsh','22','006','6',"So Joshua blessed them, and sent them away: and they went unto their tents."],
['Joshua','Jsh','22','007','7',"Now to the \emph{one} half of the tribe of Manasseh Moses had given \emph{possession} in Bashan: but unto the \emph{other} half thereof gave Joshua among their brethren on this side Jordan westward. And when Joshua sent them away also unto their tents, then he blessed them,"],
['Joshua','Jsh','22','008','8',"And he spake unto them, saying, Return with much riches unto your tents, and with very much cattle, with silver, and with gold, and with brass, and with iron, and with very much raiment: divide the spoil of your enemies with your brethren."],
['Joshua','Jsh','22','009','9',"And the children of Reuben and the children of Gad and the half tribe of Manasseh returned, and departed from the children of Israel out of Shiloh, which \emph{is} in the land of Canaan, to go unto the country of Gilead, to the land of their possession, whereof they were possessed, according to the word of the LORD by the hand of Moses."],
['Joshua','Jsh','22','010','10',"And when they came unto the borders of Jordan, that \emph{are} in the land of Canaan, the children of Reuben and the children of Gad and the half tribe of Manasseh built there an altar by Jordan, a great altar to see to."],
['Joshua','Jsh','22','011','11',"And the children of Israel heard say, Behold, the children of Reuben and the children of Gad and the half tribe of Manasseh have built an altar over against the land of Canaan, in the borders of Jordan, at the passage of the children of Israel."],
['Joshua','Jsh','22','012','12',"And when the children of Israel heard \emph{of} \emph{it}, the whole congregation of the children of Israel gathered themselves together at Shiloh, to go up to war against them."],
['Joshua','Jsh','22','013','13',"And the children of Israel sent unto the children of Reuben, and to the children of Gad, and to the half tribe of Manasseh, into the land of Gilead, Phinehas the son of Eleazar the priest,"],
['Joshua','Jsh','22','014','14',"And with him ten princes, of each chief house a prince throughout all the tribes of Israel; and each one \emph{was} an head of the house of their fathers among the thousands of Israel."],
['Joshua','Jsh','22','015','15',"And they came unto the children of Reuben, and to the children of Gad, and to the half tribe of Manasseh, unto the land of Gilead, and they spake with them, saying,"],
['Joshua','Jsh','22','016','16',"Thus saith the whole congregation of the LORD, What trespass \emph{is} this that ye have committed against the God of Israel, to turn away this day from following the LORD, in that ye have builded you an altar, that ye might rebel this day against the LORD?"],
['Joshua','Jsh','22','017','17',"\emph{Is} the iniquity of Peor too little for us, from which we are not cleansed until this day, although there was a plague in the congregation of the LORD,"],
['Joshua','Jsh','22','018','18',"But that ye must turn away this day from following the LORD? and it will be, \emph{seeing} ye rebel to day against the LORD, that to morrow he will be wroth with the whole congregation of Israel."],
['Joshua','Jsh','22','019','19',"Notwithstanding, if the land of your possession \emph{be} unclean, \emph{then} pass ye over unto the land of the possession of the LORD, wherein the LORD'S tabernacle dwelleth, and take possession among us: but rebel not against the LORD, nor rebel against us, in building you an altar beside the altar of the LORD our God."],
['Joshua','Jsh','22','020','20',"Did not Achan the son of Zerah commit a trespass in the accursed thing, and wrath fell on all the congregation of Israel? and that man perished not alone in his iniquity."],
['Joshua','Jsh','22','021','21',"Then the children of Reuben and the children of Gad and the half tribe of Manasseh answered, and said unto the heads of the thousands of Israel,"],
['Joshua','Jsh','22','022','22',"The LORD God of gods, the LORD God of gods, he knoweth, and Israel he shall know; if \emph{it} \emph{be} in rebellion, or if in transgression against the LORD, (save us not this day,)"],
['Joshua','Jsh','22','023','23',"That we have built us an altar to turn from following the LORD, or if to offer thereon burnt offering or meat offering, or if to offer peace offerings thereon, let the LORD himself require \emph{it};"],
['Joshua','Jsh','22','024','24',"And if we have not \emph{rather} done it for fear of \emph{this} thing, saying, In time to come your children might speak unto our children, saying, What have ye to do with the LORD God of Israel?"],
['Joshua','Jsh','22','025','25',"For the LORD hath made Jordan a border between us and you, ye children of Reuben and children of Gad; ye have no part in the LORD: so shall your children make our children cease from fearing the LORD."],
['Joshua','Jsh','22','026','26',"Therefore we said, Let us now prepare to build us an altar, not for burnt offering, nor for sacrifice:"],
['Joshua','Jsh','22','027','27',"But \emph{that} it \emph{may} \emph{be} a witness between us, and you, and our generations after us, that we might do the service of the LORD before him with our burnt offerings, and with our sacrifices, and with our peace offerings; that your children may not say to our children in time to come, Ye have no part in the LORD."],
['Joshua','Jsh','22','028','28',"Therefore said we, that it shall be, when they should \emph{so} say to us or to our generations in time to come, that we may say \emph{again}, Behold the pattern of the altar of the LORD, which our fathers made, not for burnt offerings, nor for sacrifices; but it \emph{is} a witness between us and you."],
['Joshua','Jsh','22','029','29',"God forbid that we should rebel against the LORD, and turn this day from following the LORD, to build an altar for burnt offerings, for meat offerings, or for sacrifices, beside the altar of the LORD our God that \emph{is} before his tabernacle."],
['Joshua','Jsh','22','030','30',"And when Phinehas the priest, and the princes of the congregation and heads of the thousands of Israel which \emph{were} with him, heard the words that the children of Reuben and the children of Gad and the children of Manasseh spake, it pleased them."],
['Joshua','Jsh','22','031','31',"And Phinehas the son of Eleazar the priest said unto the children of Reuben, and to the children of Gad, and to the children of Manasseh, This day we perceive that the LORD \emph{is} among us, because ye have not committed this trespass against the LORD: now ye have delivered the children of Israel out of the hand of the LORD."],
['Joshua','Jsh','22','032','32',"And Phinehas the son of Eleazar the priest, and the princes, returned from the children of Reuben, and from the children of Gad, out of the land of Gilead, unto the land of Canaan, to the children of Israel, and brought them word again."],
['Joshua','Jsh','22','033','33',"And the thing pleased the children of Israel; and the children of Israel blessed God, and did not intend to go up against them in battle, to destroy the land wherein the children of Reuben and Gad dwelt."],
['Joshua','Jsh','22','034','34',"And the children of Reuben and the children of Gad called the altar \emph{Ed}: for it \emph{shall} \emph{be} a witness between us that the LORD \emph{is} God."]]

Joshua023 = [
    ['Joshua','Jsh','23','001','1',"And it came to pass a long time after that the LORD had given rest unto Israel from all their enemies round about, that Joshua waxed old \emph{and} stricken in age."],
['Joshua','Jsh','23','002','2',"And Joshua called for all Israel, \emph{and} for their elders, and for their heads, and for their judges, and for their officers, and said unto them, I am old \emph{and} stricken in age:"],
['Joshua','Jsh','23','003','3',"And ye have seen all that the LORD your God hath done unto all these nations because of you; for the LORD your God \emph{is} he that hath fought for you."],
['Joshua','Jsh','23','004','4',"Behold, I have divided unto you by lot these nations that remain, to be an inheritance for your tribes, from Jordan, with all the nations that I have cut off, even unto the great sea westward."],
['Joshua','Jsh','23','005','5',"And the LORD your God, he shall expel them from before you, and drive them from out of your sight; and ye shall possess their land, as the LORD your God hath promised unto you."],
['Joshua','Jsh','23','006','6',"Be ye therefore very courageous to keep and to do all that is written in the book of the law of Moses, that ye turn not aside therefrom \emph{to} the right hand or \emph{to} the left;"],
['Joshua','Jsh','23','007','7',"That ye come not among these nations, these that remain among you; neither make mention of the name of their gods, nor cause to swear \emph{by} \emph{them}, neither serve them, nor bow yourselves unto them:"],
['Joshua','Jsh','23','008','8',"But cleave unto the LORD your God, as ye have done unto this day."],
['Joshua','Jsh','23','009','9',"For the LORD hath driven out from before you great nations and strong: but \emph{as} \emph{for} you, no man hath been able to stand before you unto this day."],
['Joshua','Jsh','23','010','10',"One man of you shall chase a thousand: for the LORD your God, he \emph{it} \emph{is} that fighteth for you, as he hath promised you."],
['Joshua','Jsh','23','011','11',"Take good heed therefore unto yourselves, that ye love the LORD your God."],
['Joshua','Jsh','23','012','12',"Else if ye do in any wise go back, and cleave unto the remnant of these nations, \emph{even} these that remain among you, and shall make marriages with them, and go in unto them, and they to you:"],
['Joshua','Jsh','23','013','13',"Know for a certainty that the LORD your God will no more drive out \emph{any} \emph{of} these nations from before you; but they shall be snares and traps unto you, and scourges in your sides, and thorns in your eyes, until ye perish from off this good land which the LORD your God hath given you."],
['Joshua','Jsh','23','014','14',"And, behold, this day I \emph{am} going the way of all the earth: and ye know in all your hearts and in all your souls, that not one thing hath failed of all the good things which the LORD your God spake concerning you; all are come to pass unto you, \emph{and} not one thing hath failed thereof."],
['Joshua','Jsh','23','015','15',"Therefore it shall come to pass, \emph{that} as all good things are come upon you, which the LORD your God promised you; so shall the LORD bring upon you all evil things, until he have destroyed you from off this good land which the LORD your God hath given you."],
['Joshua','Jsh','23','016','16',"When ye have transgressed the covenant of the LORD your God, which he commanded you, and have gone and served other gods, and bowed yourselves to them; then shall the anger of the LORD be kindled against you, and ye shall perish quickly from off the good land which he hath given unto you."]
]

Joshua024 = [
    ['Joshua','Jsh','24','001','1',"And Joshua gathered all the tribes of Israel to Shechem, and called for the elders of Israel, and for their heads, and for their judges, and for their officers; and they presented themselves before God."],
['Joshua','Jsh','24','002','2',"And Joshua said unto all the people, Thus saith the LORD God of Israel, Your fathers dwelt on the other side of the flood in old time, \emph{even} Terah, the father of Abraham, and the father of Nachor: and they served other gods."],
['Joshua','Jsh','24','003','3',"And I took your father Abraham from the other side of the flood, and led him throughout all the land of Canaan, and multiplied his seed, and gave him Isaac."],
['Joshua','Jsh','24','004','4',"And I gave unto Isaac Jacob and Esau: and I gave unto Esau mount Seir, to possess it; but Jacob and his children went down into Egypt."],
['Joshua','Jsh','24','005','5',"I sent Moses also and Aaron, and I plagued Egypt, according to that which I did among them: and afterward I brought you out."],
['Joshua','Jsh','24','006','6',"And I brought your fathers out of Egypt: and ye came unto the sea; and the Egyptians pursued after your fathers with chariots and horsemen unto the Red sea."],
['Joshua','Jsh','24','007','7',"And when they cried unto the LORD, he put darkness between you and the Egyptians, and brought the sea upon them, and covered them; and your eyes have seen what I have done in Egypt: and ye dwelt in the wilderness a long season."],
['Joshua','Jsh','24','008','8',"And I brought you into the land of the Amorites, which dwelt on the other side Jordan; and they fought with you: and I gave them into your hand, that ye might possess their land; and I destroyed them from before you."],
['Joshua','Jsh','24','009','9',"Then Balak the son of Zippor, king of Moab, arose and warred against Israel, and sent and called Balaam the son of Beor to curse you:"],
['Joshua','Jsh','24','010','10',"But I would not hearken unto Balaam; therefore he blessed you still: so I delivered you out of his hand."],
['Joshua','Jsh','24','011','11',"And ye went over Jordan, and came unto Jericho: and the men of Jericho fought against you, the Amorites, and the Perizzites, and the Canaanites, and the Hittites, and the Girgashites, the Hivites, and the Jebusites; and I delivered them into your hand."],
['Joshua','Jsh','24','012','12',"And I sent the hornet before you, which drave them out from before you, \emph{even} the two kings of the Amorites; \emph{but} not with thy sword, nor with thy bow."],
['Joshua','Jsh','24','013','13',"And I have given you a land for which ye did not labour, and cities which ye built not, and ye dwell in them; of the vineyards and oliveyards which ye planted not do ye eat."],
['Joshua','Jsh','24','014','14',"Now therefore fear the LORD, and serve him in sincerity and in truth: and put away the gods which your fathers served on the other side of the flood, and in Egypt; and serve ye the LORD."],
['Joshua','Jsh','24','015','15',"And if it seem evil unto you to serve the LORD, choose you this day whom ye will serve; whether the gods which your fathers served that \emph{were} on the other side of the flood, or the gods of the Amorites, in whose land ye dwell: but as for me and my house, we will serve the LORD."],
['Joshua','Jsh','24','016','16',"And the people answered and said, God forbid that we should forsake the LORD, to serve other gods;"],
['Joshua','Jsh','24','017','17',"For the LORD our God, he \emph{it} \emph{is} that brought us up and our fathers out of the land of Egypt, from the house of bondage, and which did those great signs in our sight, and preserved us in all the way wherein we went, and among all the people through whom we passed:"],
['Joshua','Jsh','24','018','18',"And the LORD drave out from before us all the people, even the Amorites which dwelt in the land: \emph{therefore} will we also serve the LORD; for he \emph{is} our God."],
['Joshua','Jsh','24','019','19',"And Joshua said unto the people, Ye cannot serve the LORD: for he \emph{is} an holy God; he \emph{is} a jealous God; he will not forgive your transgressions nor your sins."],
['Joshua','Jsh','24','020','20',"If ye forsake the LORD, and serve strange gods, then he will turn and do you hurt, and consume you, after that he hath done you good."],
['Joshua','Jsh','24','021','21',"And the people said unto Joshua, Nay; but we will serve the LORD."],
['Joshua','Jsh','24','022','22',"And Joshua said unto the people, Ye \emph{are} witnesses against yourselves that ye have chosen you the LORD, to serve him. And they said, \emph{We} \emph{are} witnesses."],
['Joshua','Jsh','24','023','23',"Now therefore put away, \emph{said} \emph{he}, the strange gods which \emph{are} among you, and incline your heart unto the LORD God of Israel."],
['Joshua','Jsh','24','024','24',"And the people said unto Joshua, The LORD our God will we serve, and his voice will we obey."],
['Joshua','Jsh','24','025','25',"So Joshua made a covenant with the people that day, and set them a statute and an ordinance in Shechem."],
['Joshua','Jsh','24','026','26',"And Joshua wrote these words in the book of the law of God, and took a great stone, and set it up there under an oak, that \emph{was} by the sanctuary of the LORD."],
['Joshua','Jsh','24','027','27',"And Joshua said unto all the people, Behold, this stone shall be a witness unto us; for it hath heard all the words of the LORD which he spake unto us: it shall be therefore a witness unto you, lest ye deny your God."],
['Joshua','Jsh','24','028','28',"So Joshua let the people depart, every man unto his inheritance."],
['Joshua','Jsh','24','029','29',"And it came to pass after these things, that Joshua the son of Nun, the servant of the LORD, died, \emph{being} an hundred and ten years old."],
['Joshua','Jsh','24','030','30',"And they buried him in the border of his inheritance in Timnath-serah, which \emph{is} in mount Ephraim, on the north side of the hill of Gaash."],
['Joshua','Jsh','24','031','31',"And Israel served the LORD all the days of Joshua, and all the days of the elders that overlived Joshua, and which had known all the works of the LORD, that he had done for Israel."],
['Joshua','Jsh','24','032','32',"And the bones of Joseph, which the children of Israel brought up out of Egypt, buried they in Shechem, in a parcel of ground which Jacob bought of the sons of Hamor the father of Shechem for an hundred pieces of silver: and it became the inheritance of the children of Joseph."],
['Joshua','Jsh','24','033','33',"And Eleazar the son of Aaron died; and they buried him in a hill \emph{that} \emph{pertained} \emph{to} Phinehas his son, which was given him in mount Ephraim."]
]

def main():
    ProcessVerses('Joshua018',Joshua018)


if __name__ == "__main__":
    main()




%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And the whole congregation of the children of Israel assembled together at Shiloh, and set up the tabernacle of the congregation there. And the land was subdued before them.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And there remained among the children of Israel seven tribes, which had not yet received their inheritance.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And Joshua said unto the children of Israel, How long \emph{are} ye slack to go to possess the land, which the LORD God of your fathers hath given you?}
[4] \textcolor[cmyk]{0.99998,1,0,0}{Give out from among you three men for \emph{each} tribe: and I will send them, and they shall rise, and go through the land, and describe it according to the inheritance of them; and they shall come \emph{again} to me.}
[5] \textcolor[cmyk]{

cast & 3 \\ \hline
charged & 1 \\ \hline
children & 11 \\ \hline
cities & 4 \\ \hline
city & 1 \\ \hline
coast & 3 \\ \hline
coasts & 2 \\ \hline
come & 2 \\ \hline
compassed & 1 \\ \hline
congregation & 2 \\ \hline
corner & 1 \\ \hline
descended & 4 \\ \hline
describe & 4 \\ \hline
described & 1 \\ \hline
divide & 1 \\ \hline
divided & 1 \\ \hline
divisions & 1 \\ \hline
down & 2 \\ \hline
drawn & 2 \\ \hline
east & 2 \\ \hline
end & 3 \\ \hline
families & 4 \\ \hline
fathers & 1 \\ \hline
for & 5 \\ \hline
forth & 3 \\ \hline
fourteen & 1 \\ \hline
from & 6 \\ \hline
gave & 1 \\ \hline
giants & 1 \\ \hline
given & 1 \\ \hline
go & 2 \\ \hline
going & 1 \\ \hline
goings & 2 \\ \hline
had & 1 \\ \hline
half & 1 \\ \hline
hath & 1 \\ \hline
have & 2 \\ \hline
here & 2 \\ \hline
hill & 2 \\ \hline
hither & 1 \\ \hline
host & 1 \\ \hline
house & 1 \\ \hline
in & 6 \\ \hline
inheritance & 6 \\ \hline
into & 2 \\ \hline
it & 5 \\ \hline
land & 8 \\ \hline
long & 1 \\ \hline
lot & 2 \\ \hlin